In [1]:
!pip install openpyxl reportlab sentence-transformers unidecode
!python -m spacy download es_core_news_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
# --- Imports Básicos ---
import pandas as pd
import numpy as np
import io
import re
import logging
import torch
from datetime import datetime
from enum import Enum
from unidecode import unidecode
from sentence_transformers import SentenceTransformer, util
import spacy

# Librerías para UI en Colab
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Para forzar descarga de archivos en Colab
from google.colab import files

# Para generar Excel
# Asegúrate de que openpyxl esté instalado: !pip install openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

# Para generar PDF
# Asegúrate de que reportlab esté instalado: !pip install reportlab
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors as reportlab_colors
from reportlab.platypus import (
    SimpleDocTemplate,
    Paragraph,
    Spacer,
    Image as ReportlabImage,
    Table,
    TableStyle,
    PageBreak,
    KeepInFrame
)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_JUSTIFY
from reportlab.lib.units import inch
from reportlab.pdfbase import ttfonts, pdfmetrics

# ### CORRECCIÓN: La siguiente línea que causaba el ImportError ha sido eliminada ###
# from reportlab.pdfbase.pdfdoc import UnicodeString

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Cargar modelo spaCy para español (si está disponible)
# Asegúrate de descargar el modelo: !python -m spacy download es_core_news_lg
try:
    nlp = spacy.load("es_core_news_lg")
    SPACY_AVAILABLE = True
    logging.info("Modelo spaCy 'es_core_news_lg' cargado correctamente.")
except OSError:
    nlp = None
    SPACY_AVAILABLE = False
    logging.warning("No se pudo cargar 'es_core_news_lg'. La lematización estará desactivada.")

# --- Constantes de la Lógica ---
SALARY_DIFF_PERCENTAGE_THRESHOLD = 0.10  # 10%
GRADE_DIFF_THRESHOLD = 0  # Mismo grado exacto (sin diferencia)
DEFAULT_UMBRAL_DECISION_FUNCIONES = 0.60
DEFAULT_UMBRAL_DECISION_REQUISITOS_COMP = 0.65 # Umbral para Competencias y Experiencia (informativo)
DEFAULT_UMBRAL_DECISION_REQUISITOS_ESTUDIO = 0.70 # Nuevo umbral para Requisitos de Estudio
DEFAULT_UMBRAL_CONEXION = 0.65
DEFAULT_UMBRAL_COBERTURA_BIN = 0.60
DEFAULT_PESO_COSENO = 1.00
DEFAULT_PESO_JACCARD = 0.00
DEFAULT_TOP_N = 3 # Cantidad de mejores coincidencias a mostrar en detalles y PDF

# Dimensiones máximas para visualizaciones y reportes
MIN_WORDS_PER_ITEM = 2
MAX_TEXT_LEN_VIZ = 50
MAX_TEXT_LEN_PDF = 60
MAX_ITEMS_FOR_HEATMAP_ANNOT = 400 # Limita las anotaciones en heatmaps muy grandes

# Columna identificadora principal en el Excel
ID_COLUMN_NAME = "No. OPEC"

# Pesos por defecto para la decisión final automatizada (deben sumar 1.0)
DEFAULT_WEIGHT_FUNCIONES = 0.80
DEFAULT_WEIGHT_ESTUDIO = 0.20
DEFAULT_WEIGHT_COMP_LAB = 0.00
DEFAULT_WEIGHT_COMP_COMP = 0.00


# --- Enumeraciones ---
class TipoUsoEstudio(Enum):
    INCORPORACION = "Incorporación (en la misma entidad)"
    REINCORPORACION = "Reincorporación (puede ser en entidades distintas)"
    REUBICACION_VIOLENCIA = "Reubicación (Traslado) por razones de violencia"
    TRASLADO = "Traslado"
    OTRO = "Otro (especificar)"

LISTA_TIPOS_USO = [uso.value for uso in TipoUsoEstudio]

class TipoExperiencia(Enum):
    NO_IDENTIFICADO = "No Identificado"
    LABORAL = "Laboral"
    LABORAL_RELACIONADA = "Laboral Relacionada"
    PROFESIONAL = "Profesional"
    PROFESIONAL_RELACIONADA = "Profesional Relacionada"
    DOCENTE = "Docente"

# Jerarquía de Experiencia: qué tipo de experiencia (clave) puede satisfacer un requisito (valores en el set).
# Basado en la lógica: la experiencia "superior" o más específica puede cumplir roles más generales.
# Profesional Relacionada -> Profesional -> Laboral
# Docente -> Profesional -> Laboral
HIERARQUIA_EXPERIENCIA = {
    TipoExperiencia.PROFESIONAL_RELACIONADA: {
        TipoExperiencia.PROFESIONAL_RELACIONADA,
        TipoExperiencia.PROFESIONAL,
        TipoExperiencia.LABORAL_RELACIONADA,
        TipoExperiencia.LABORAL
    },
    TipoExperiencia.PROFESIONAL: {
        TipoExperiencia.PROFESIONAL,
        TipoExperiencia.LABORAL
    },
    TipoExperiencia.LABORAL_RELACIONADA: {
        TipoExperiencia.LABORAL_RELACIONADA,
        TipoExperiencia.LABORAL
    },
    TipoExperiencia.DOCENTE: {
        TipoExperiencia.DOCENTE,
        TipoExperiencia.PROFESIONAL,
        TipoExperiencia.LABORAL
    },
    TipoExperiencia.LABORAL: {
        TipoExperiencia.LABORAL
    },
    TipoExperiencia.NO_IDENTIFICADO: {
        TipoExperiencia.NO_IDENTIFICADO # Solo puede satisfacerse a sí mismo
    }
}


class AppColors(Enum):
    PRIMARY = reportlab_colors.HexColor('#90caf9')      # Azul claro
    SECONDARY = reportlab_colors.HexColor('#f48fb1')    # Rosa claro
    SUCCESS = reportlab_colors.HexColor('#66bb6a')      # Verde claro
    DANGER = reportlab_colors.HexColor('#e57373')       # Rojo claro
    WARNING = reportlab_colors.HexColor('#ffb74d')      # Naranja claro
    INFO = reportlab_colors.HexColor('#4fc3f7')         # Celeste claro
    GRID = reportlab_colors.HexColor('#424242')         # Gris oscuro para líneas (HTML)
    BACKGROUND = reportlab_colors.HexColor('#000000')   # NEGRO para PDF (original, será cambiado para PDF)
    TEXT_DARK = reportlab_colors.HexColor('#ffffff')    # Blanco para texto (HTML)
    TEXT_LIGHT = reportlab_colors.HexColor('#bdbdbd')   # Gris claro (HTML)
    ACCENT_BACKGROUND = reportlab_colors.HexColor('#212121') # Fondo de acento oscuro (HTML)
    ACCENT_BORDER = reportlab_colors.HexColor('#616161') # Borde de acento oscuro (HTML)
    WHITE_SMOKE = reportlab_colors.whitesmoke # Usado en PDF, es claro
    LIGHT_GREY = reportlab_colors.lightgrey # Usado en PDF, es claro
    NEUTRAL = reportlab_colors.grey # Gris neutro

class ModelOptions(Enum):
    MPNET_MULTI = 'paraphrase-multilingual-mpnet-base-v2'
    STS_ES = 'hiiamsid/sentence_similarity_spanish_es'

MODEL_DESCRIPTIONS = {
    ModelOptions.MPNET_MULTI.value: 'MPNet Multilingüe v2 (SBERT)',
    ModelOptions.STS_ES.value: 'STS RoBERTa-BNE (Español)',
}

DEFAULT_MODEL_ORDER = [
    ModelOptions.STS_ES.value,
    ModelOptions.MPNET_MULTI.value,
]

# --- Funciones Auxiliares de Preprocesamiento de Texto ---

def _is_valid_data(data_str: str) -> bool:
    """Verifica si la cadena de datos es válida (no vacía y no 'nan')."""
    val = str(data_str).strip().lower()
    return bool(val and val != 'nan')

def remover_ordinales(line: str) -> str:
    """Elimina prefijos comunes de viñetas u ordinales."""
    pattern = r'^\s*(?:[\[\(]?\d+[.\)\-\]]?|[\[\(]?[a-zA-Z][.\)\-\]]|[IVXLCDMivxlcdm]+[.\)\-\]]?|[•*\-\+])\s*'
    return re.sub(pattern, '', line).strip()

def normalizar_texto_func(txt: str) -> str:
    """Convierte a minúsculas, quita acentos y espacios extra."""
    if not isinstance(txt, str): return ""
    return unidecode(txt.strip().lower())

def es_funcion_comodin(linea: str) -> bool:
    """
    Verifica si una línea es un comodín genérico de funciones de forma más robusta.
    Esta función ha sido mejorada para detectar con mayor precisión frases "basura".
    """
    check = normalizar_texto_func(linea)

    # Frases que, si aparecen en cualquier parte, invalidan la línea.
    # Son frases que no suelen tener continuación útil.
    comodines_contenidos = [
        "demas funciones", "demas labores", "demas asignadas por",
        "demas que le sean asignadas", "y funciones similares",
        "otras funciones asignadas", "lo que le sea asignado por",
        "asignadas por el jefe inmediato", "asignadas por la ley",
        "funciones inherentes al cargo", "propio de su cargo", "propias del cargo",
        "actividades que le sean delegadas", "cumplir con las demas funciones"
    ]

    # Frases que suelen estar al principio de un item comodín.
    comodines_inicio = [
        "las demas", "y las demas", "y demas", "las otras", "otras que",
        "atender las demas", "colaborar en las demas", "apoyar en las demas"
    ]

    # Palabras sueltas que indican genericidad, sobre todo en frases cortas.
    comodines_palabras = ["etc", "etc.", "demas"]

    for c in comodines_contenidos:
        if c in check:
            return True

    for c in comodines_inicio:
        if check.startswith(c):
            return True

    palabras = check.split()
    if len(palabras) <= 5: # Un poco más de margen para frases cortas
        for p in palabras:
            if p in comodines_palabras:
                return True

    return False

def es_ruido(linea: str, min_words: int = MIN_WORDS_PER_ITEM, es_funcion: bool = False) -> bool:
    """Determina si una línea es ruido: vacía, muy corta o comodín (si es función)."""
    linea_stripped = linea.strip()
    if not linea_stripped:
        return True
    palabras = remover_ordinales(linea_stripped).split()
    if len(palabras) < min_words:
        return True
    if es_funcion and es_funcion_comodin(linea_stripped):
        return True
    return False

def lematizar_texto_func(txt: str, nlp_model: spacy.language.Language) -> str:
    """Lematiza un texto usando spaCy, si está disponible."""
    if not nlp_model or not txt:
        return txt
    try:
        if len(txt) > nlp_model.max_length:
            nlp_model.max_length = len(txt) + 100
        doc = nlp_model(txt)
        lemmas = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space and token.lemma_.strip()]
        return " ".join(lemmas)
    except Exception as e:
        logging.error(f"Fallo lematizar '{txt[:30]}...': {e}")
        return txt

def preprocesar_educacion(texto: str) -> str:
    """Preprocesamiento específico para requisitos de Educación (eliminar frases introductorias)."""
    if not isinstance(texto, str):
        return ""
    texto_proc = texto.lower()
    frases_a_eliminar = [
        # Frases introductorias de títulos/formación (más agresivo)
        r"^\s*t[íi]tulo\s+(?:profesional|de\s+formaci[óo]n)\s+(?:universitaria|tecnol[óo]gica|t[ée]cnica\s+profesional)\s*(?:en|en\s+la\s+disciplina\s+acad[ée]mica\s+de|en\s+el\s+[áa]rea\s+de|en\s+la\s+modalidad\s+de)?\s*\b",
        r"^\s*diploma\s+(?:de\s+bachiller|universitario|profesional)\s*(?:en)?\s*\b",
        r"^\s*(?:pregrado|posgrado|maestr[íi]a|especializaci[óo]n|doctorado)\s*(?:en\s+el\s+[áa]rea\s+de|en)?\s*\b",
        r"^\s*estudios\s+(?:profesionales?|universitarios?|t[ée]cnicos?|tecnol[óo]gicos?|de\s+posgrado|de\s+maestr[íi]a|de\s+especializaci[óo]n|de\s+doctorado)\s*(?:en)?\s*\b",
        r"^\s*formaci[óo]n\s+(?:profesional|universitaria|t[ée]nica|tecnol[óo]gica|de\s+posgrado|acad[ée]mica)\s*(?:en)?\s*\b",
        r"^\s*requiere\s+(?:t[íi]tulo|estudios?|formaci[óo]n)\s*(?:en)?\s*\b",
        # Frases comunes de posgrado
        r"^\s*(?:t[íi]tulo|certificado|diploma)\s+de\s+posgrado\s+(?:en\s+la\s+modalidad\s+de)?\s*(?:especializaci[óo]n|maestr[íi]a|doctorado)\s*(?:en)?\s*\b",
        # Menciones a tarjetas/matrículas profesionales
        r"\b(?:y|con|y\s+con)\s+(?:tarjeta|matr[íi]cula|licencia)\s+profesional\s+(?:vigente|activa|correspondiente|requerida|expedida).*",
        r"\b(?:tarjeta|matr[íi]cula|licencia)\s+profesional\b.*$",
        r"^\s*(?:tarjeta|matr[íi]cula)\s+profesional.*",
        # Menciones a regulaciones
        r"\b(?:en\s+los\s+casos\s+(?:de\s+ley|reglamentados\s+por\s+la\s+ley))\b",
        # Menciones a Núcleo Básico del Conocimiento (NBC)
        r"^\s*n[úu]cleo\s+b[áa]sico\s+del\s+conocimiento\s*[:\s-]*\s*(?:en)?\s*",
        r"^\s*disciplina\s+acad[ée]mica\s*[:\s-]*\s*(?:en)?\s*",
        r"^\s*[áa]rea\s+del\s+conocimiento\s*[:\s-]*\s*(?:en)?\s*",
        r"^\s*nbc\s*[:\s-]*\s*", # NBC solo o al inicio de línea
        # Viñetas o numeración residual
        r"^\s*[a-z]\)\s*", # e.g., a)
        r"^\s*\d+\.\s*",   # e.g., 1.
    ]
    for pattern in frases_a_eliminar:
        texto_proc = re.sub(pattern, '', texto_proc, flags=re.IGNORECASE | re.MULTILINE).strip()

    # Eliminar líneas que solo contienen "nbc" o "n.b.c."
    texto_proc = re.sub(r"^\s*(nbc|n\.b\.c\.)\s*$", "", texto_proc, flags=re.IGNORECASE | re.MULTILINE).strip()

    # Consolidar múltiples espacios en uno solo
    texto_proc = re.sub(r'\s+', ' ', texto_proc).strip()

    lineas = [line.strip() for line in texto_proc.splitlines() if line.strip()]
    return "\n".join(lineas)

# NUEVA FUNCIÓN para extraer tipo de experiencia
def extraer_tipo_y_texto_experiencia(texto: str) -> tuple[TipoExperiencia, str]:
    """
    Extrae el tipo de experiencia y el texto descriptivo limpio.
    Busca los términos más específicos primero.
    """
    if not isinstance(texto, str):
        return TipoExperiencia.NO_IDENTIFICADO, ""

    texto_norm = normalizar_texto_func(texto)
    texto_original_limpio = texto

    # El orden es crucial: de más específico a más general
    tipos_regex = {
        TipoExperiencia.PROFESIONAL_RELACIONADA: r"experiencia\s+profesional\s+relacionada",
        TipoExperiencia.LABORAL_RELACIONADA: r"experiencia\s+laboral\s+relacionada",
        TipoExperiencia.PROFESIONAL: r"experiencia\s+profesional",
        TipoExperiencia.DOCENTE: r"experiencia\s+docente",
        TipoExperiencia.LABORAL: r"experiencia\s+laboral",
    }

    for tipo, pattern in tipos_regex.items():
        match = re.search(pattern, texto_norm)
        if match:
            # Remover la frase del tipo de experiencia para la comparación semántica
            texto_original_limpio = re.sub(pattern, '', texto, flags=re.IGNORECASE).strip()
            return tipo, texto_original_limpio

    # Si no se encontró un tipo específico, puede ser que solo diga "experiencia"
    if "experiencia" in texto_norm:
        # Por defecto, se asume 'Laboral' si no se especifica más.
        return TipoExperiencia.LABORAL, texto

    return TipoExperiencia.NO_IDENTIFICADO, texto

# NUEVA FUNCIÓN para verificar la jerarquía de experiencia
def verificar_jerarquia_experiencia(
    tipo_base: TipoExperiencia,
    tipo_destino_requerido: TipoExperiencia
) -> tuple[bool, str]:
    """
    Verifica si el tipo de experiencia del empleo base satisface el requisito del empleo destino.
    """
    if tipo_destino_requerido == TipoExperiencia.NO_IDENTIFICADO or tipo_base == TipoExperiencia.NO_IDENTIFICADO:
        return True, "No se identificó un tipo de experiencia específico; se omite la verificación de jerarquía."

    aceptables = HIERARQUIA_EXPERIENCIA.get(tipo_base, set())

    if tipo_destino_requerido in aceptables:
        return True, f"Jerarquía Válida: La experiencia '{tipo_base.value}' del Empleo Base es aceptable para el requisito de '{tipo_destino_requerido.value}' del Empleo Destino."
    else:
        return False, f"Jerarquía Inválida: La experiencia '{tipo_base.value}' del Empleo Base NO es aceptable para el requisito de '{tipo_destino_requerido.value}' del Empleo Destino."


def split_by_bullet_lines(text: str) -> list[str]:
    """Divide un bloque de texto en líneas/ítems basado en viñetas o numeración."""
    # Esta expresión regular es más robusta para manejar diferentes formatos de viñetas.
    bullet_pattern = re.compile(r'^\s*(?:[•\-*+]|\d+[.)]|[\[\(]?\d+[.\)\-\]]?|[a-zA-Z][.)]|[\[\(]?[a-zA-Z][.\)\-\]]?)\s*')
    lines = text.splitlines()
    items = []
    current_item_lines: list[str] = []

    for line in lines:
        line_stripped = line.strip()
        if not line_stripped:
            continue

        is_bullet_start = bullet_pattern.match(line)

        if is_bullet_start:
            if current_item_lines:
                items.append(" ".join(current_item_lines).strip())
            clean_line = bullet_pattern.sub('', line).strip()
            current_item_lines = [clean_line] if clean_line else []
        else:
            if current_item_lines:
                current_item_lines.append(line_stripped)
            # Heurística: Si no es viñeta pero ya hay items, podría ser una continuación
            # o un nuevo item sin viñeta. Lo tratamos como continuación.
            elif items:
                items[-1] = f"{items[-1]} {line_stripped}"
            else:
                current_item_lines.append(line_stripped)

    if current_item_lines:
        items.append(" ".join(current_item_lines).strip())

    # Fallback si no hay viñetas pero sí hay saltos de línea significativos.
    if not items and lines:
        non_empty_lines = [l.strip() for l in lines if l.strip()]
        # Consideramos cada línea no vacía como un ítem.
        items = [remover_ordinales(f) for f in non_empty_lines]

    return [f for f in items if f]


def procesar_bloque(
    texto: str,
    aspecto: str,
    eliminar_duplicados: bool,
    normalizar: bool,
    lematizar: bool,
    nlp_model: spacy.language.Language | None,
    min_words: int
) -> list[str]:
    """Procesa un bloque de texto para extraer items significativos."""
    if not _is_valid_data(texto):
        return []

    # El preprocesamiento especial ahora se controla desde fuera para 'Experiencia'
    texto_preprocesado = texto
    if aspecto == 'Educacion':
        texto_preprocesado = preprocesar_educacion(texto)
        if not _is_valid_data(texto_preprocesado):
            return []

    items_crudos = split_by_bullet_lines(texto_preprocesado)
    items_procesados: list[str] = []
    vistos = set()

    for item in items_crudos:
        if es_ruido(item, min_words, es_funcion=(aspecto == 'Funciones')):
            continue

        item_for_processing = item
        if normalizar:
            item_for_processing = normalizar_texto_func(item_for_processing)

        if lematizar and nlp_model:
            item_for_processing = lematizar_texto_func(item_for_processing, nlp_model)

        if not item_for_processing or len(item_for_processing.split()) < min_words:
            continue

        clave_duplicado = item_for_processing
        item_a_guardar = item # Por defecto, guardar el original

        if normalizar and not lematizar :
                item_a_guardar = normalizar_texto_func(item)
        elif lematizar:
                item_a_guardar = item_for_processing

        if eliminar_duplicados:
            if clave_duplicado not in vistos:
                items_procesados.append(item_a_guardar)
                vistos.add(clave_duplicado)
        else:
            items_procesados.append(item_a_guardar)

    return items_procesados

# --- Funciones de Similitud ---

def jaccard_similarity(texto_a: str, texto_b: str) -> float:
    """Calcula similitud Jaccard entre dos textos."""
    set_a = set(texto_a.split())
    set_b = set(texto_b.split())
    if not set_a and not set_b:
        return 1.0
    inter = set_a.intersection(set_b)
    union = set_a.union(set_b)
    if not union:
        return 0.0
    return len(inter) / len(union)

def calcular_matriz_similitud_combinada(
    items_1: list[str],
    items_2: list[str],
    model: SentenceTransformer,
    device: str,
    usar_jaccard: bool,
    peso_coseno: float,
    peso_jaccard: float
) -> np.ndarray:
    """Genera la matriz de similitud combinada (coseno + Jaccard opcional)."""
    num_1 = len(items_1)
    num_2 = len(items_2)
    matriz = np.zeros((num_1, num_2))

    if not items_1 or not items_2 or not model:
        return matriz

    if usar_jaccard and (peso_coseno + peso_jaccard > 1e-9):
        suma_pesos = peso_coseno + peso_jaccard
        w_cos = peso_coseno / suma_pesos
        w_jacc = peso_jaccard / suma_pesos
    else:
        w_cos = 1.0
        w_jacc = 0.0
        usar_jaccard = False

    try:
        if not items_1 or not items_2:
            return np.zeros((num_1, num_2))

        emb_1 = model.encode(items_1, convert_to_tensor=True, device=device, show_progress_bar=False)
        emb_2 = model.encode(items_2, convert_to_tensor=True, device=device, show_progress_bar=False)

        if emb_1.nelement() == 0 or emb_2.nelement() == 0:
            logging.warning("Embeddings result in empty tensors after encoding. Check input items.")
            return np.zeros((num_1, num_2))

        matriz_cos = util.cos_sim(emb_1, emb_2).cpu().numpy()
        matriz_cos = np.clip(matriz_cos, 0.0, 1.0)

        if usar_jaccard and w_jacc > 1e-9 :
            matriz_jac = np.zeros((num_1, num_2), dtype=np.float32)
            for i in range(num_1):
                for j in range(num_2):
                    if items_1[i] and items_2[j]:
                        matriz_jac[i, j] = jaccard_similarity(items_1[i], items_2[j])
            matriz = (w_cos * matriz_cos) + (w_jacc * matriz_jac)
        else:
            matriz = matriz_cos

        return np.clip(matriz, 0.0, 1.0)

    except Exception as e:
        logging.error(f"Error al calcular similitud: {e}", exc_info=True)
        return np.zeros((num_1, num_2))


def calcular_metricas_agregadas(matriz: np.ndarray) -> dict[str, float]:
    """Calcula métricas agregadas (cobertura promedio y similitud promedio total)."""
    metricas: dict[str, float] = {
        'cobertura_prom_max_1': 0.0,
        'cobertura_prom_max_2': 0.0,
        'sim_promedio_total': 0.0
    }
    if matriz.ndim == 2 and matriz.size > 0:
        try:
            max_filas = np.max(matriz, axis=1)
            if max_filas.size > 0:
                metricas['cobertura_prom_max_1'] = float(np.mean(max_filas))

            max_cols = np.max(matriz, axis=0)
            if max_cols.size > 0:
                metricas['cobertura_prom_max_2'] = float(np.mean(max_cols))

            metricas['sim_promedio_total'] = float(np.mean(matriz))
        except Exception as e:
            logging.error(f"Error métricas agregadas: {e}")
    return metricas

def calcular_cobertura_binaria(matriz: np.ndarray, umbral: float, axis: int = 1) -> float:
    """Calcula porcentaje de items cuyo máximo ≥ umbral en filas (axis=1) o columnas (axis=0)."""
    if matriz.ndim != 2 or matriz.size == 0:
        return 0.0
    try:
        maximos = np.max(matriz, axis=axis)
        if maximos.size == 0:
            return 0.0
        return float(np.mean(maximos >= umbral))
    except Exception as e:
        logging.error(f"Error cobertura binaria axis={axis}: {e}")
        return 0.0

def obtener_nivel_similitud(cobertura: float) -> tuple[str, str]:
    """Clasifica nivel de similitud en función de cobertura promedio."""
    if cobertura >= 0.8:
        return "Alta (≥80%)", AppColors.SUCCESS.value.hexval()
    elif cobertura >= 0.6:
        return "Media-Alta (60%-79%)", AppColors.PRIMARY.value.hexval()
    elif cobertura >= 0.4:
        return "Parcial (40%-59%)", AppColors.WARNING.value.hexval()
    else:
        return "Baja (<40%)", AppColors.DANGER.value.hexval()

def reordenar_columnas_por_similitud(
    matriz: np.ndarray,
    items_2: list[str]
) -> tuple[np.ndarray, list[str], list[int]]:
    """Reordena columnas de la matriz y lista items_2 según similitud promedio descendente."""
    if matriz.ndim != 2 or matriz.size == 0 or len(items_2) != matriz.shape[1]:
        return matriz, items_2, list(range(len(items_2)))
    try:
        promedios_columna = np.mean(matriz, axis=0)
        indices_reordenados = np.argsort(promedios_columna)[::-1]

        matriz_reordenada = matriz[:, indices_reordenados]
        items2_reordenados = [items_2[i] for i in indices_reordenados]

        return matriz_reordenada, items2_reordenados, list(indices_reordenados)
    except Exception as e:
        logging.error(f"Error reordenar columnas: {e}", exc_info=True)
        return matriz, items_2, list(range(len(items_2)))


def truncar_texto(texto: str | None, max_len: int) -> str:
    """Trunca un texto a max_len caracteres, agregando '...' si excede."""
    if not isinstance(texto, str):
        return ""
    return texto[:max_len] + "..." if len(texto) > max_len else texto

# --- Visualizaciones (utilizadas en Colab y PDF) ---

def generar_heatmap(
    matriz: np.ndarray,
    items_1: list[str],
    items_2: list[str],
    titulo: str = "Matriz de Similitud",
    label_1: str = "Items E1",
    label_2: str = "Items E2 (Reordenados)",
    prefix_1: str = "E1",
    prefix_2: str = "E2",
    umbral_annot: float = 0.4,
    cmap: str = "Blues",
    label_cbar: str = "Similitud"
) -> io.BytesIO | None:
    """Genera un heatmap PNG en buffer de bytes (Light Theme for PDF/Colab)."""
    if matriz.ndim != 2 or matriz.size == 0 or not items_1 or not items_2:
        logging.warning("Heatmap: Datos insuficientes o inválidos.")
        return None
    num_1, num_2 = matriz.shape
    if num_1 == 0 or num_2 == 0 or num_1 != len(items_1) or num_2 != len(items_2):
        logging.warning(f"Heatmap: Discrepancia de dimensiones. Matriz: {matriz.shape}, Items1: {len(items_1)}, Items2: {len(items_2)}")
        return None

    import matplotlib.pyplot as plt
    import seaborn as sns

    # Dynamic figure size to better accommodate labels
    fig_width = max(8, min(25, num_2 * 0.8 if num_2 > 0 else 8))
    fig_height = max(6, min(30, num_1 * 0.6 if num_1 > 0 else 6))
    fontsize = 9 if max(num_1, num_2) < 20 else 8

    # Adjust font size for annotations based on number of cells
    annot_fontsize = 8
    if num_1 * num_2 > 200: annot_fontsize = 6
    if num_1 * num_2 > 500: annot_fontsize = 5

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(fig_width, fig_height), dpi=120)

    labels_1 = [f"{prefix_1}-{i+1}: {truncar_texto(f, MAX_TEXT_LEN_VIZ)}" for i, f in enumerate(items_1)]
    labels_2 = [f"{prefix_2}-{j+1}: {truncar_texto(f, MAX_TEXT_LEN_VIZ)}" for j, f in enumerate(items_2)]

    show_annot = (num_1 * num_2) < MAX_ITEMS_FOR_HEATMAP_ANNOT
    annot_data = matriz if show_annot else False
    annot_mask = (matriz < umbral_annot) if show_annot else None

    plot_text_color = 'black'
    plot_bg_color = 'white'
    plot_grid_color = 'lightgrey'

    try:
        sns.heatmap(
            matriz,
            annot=annot_data,
            fmt=".2f",
            cmap=cmap,
            xticklabels=labels_2,
            yticklabels=labels_1,
            linewidths=0.5,
            linecolor=plot_grid_color,
            cbar=True,
            cbar_kws={'label': label_cbar, 'shrink': 0.7},
            mask=annot_mask,
            annot_kws={"size": annot_fontsize, "color": plot_text_color},
            ax=ax,
            vmin=0.0,
            vmax=1.0
        )
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize, color=plot_text_color)
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=fontsize, color=plot_text_color)
        # MEJORA: Se convierte explícitamente a string para evitar errores sutiles de renderizado.
        ax.set_title(str(titulo), fontsize=fontsize + 3, pad=20, color=plot_text_color)
        ax.set_xlabel(label_2, fontsize=fontsize + 1, color=plot_text_color, labelpad=15)
        ax.set_ylabel(label_1, fontsize=fontsize + 1, color=plot_text_color, labelpad=15)

        fig.patch.set_facecolor(plot_bg_color)
        ax.set_facecolor(plot_bg_color)

        cbar_ax = fig.axes[-1]
        cbar_ax.yaxis.label.set_color(plot_text_color)
        cbar_ax.tick_params(axis='y', colors=plot_text_color)

        fig.tight_layout(rect=[0, 0.03, 1, 0.95])

        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight', facecolor=fig.get_facecolor())
        buf.seek(0)
        plt.close(fig)
        return buf
    except Exception as e:
        logging.error(f"Error generar heatmap: {e}", exc_info=True)
        if 'fig' in locals() and fig:
            plt.close(fig)
        return None

def generar_diagrama_conexiones(
    items_1: list[str],
    items_2: list[str],
    matriz: np.ndarray,
    titulo: str = "Diagrama de Conexiones Semánticas",
    label_1: str = "Empleo 1",
    label_2: str = "Items E2 (Reordenados)",
    prefix_1: str = "E1",
    prefix_2: str = "E2",
    umbral_conexion: float = DEFAULT_UMBRAL_CONEXION,
    cmap: str = "Blues"
) -> io.BytesIO | None:
    """Genera diagrama de conexiones en buffer PNG (Light Theme)."""
    if matriz.ndim != 2 or matriz.size == 0 or not items_1 or not items_2:
        logging.warning("Diagrama Conexiones: Datos insuficientes o inválidos.")
        return None
    num_1, num_2 = matriz.shape
    if num_1 != len(items_1) or num_2 != len(items_2):
        logging.warning(f"Diagrama Conexiones: Discrepancia de dimensiones. Matriz: {matriz.shape}, Items1: {len(items_1)}, Items2: {len(items_2)}")
        return None

    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors

    fig_height = max(6, min(20, max(num_1, num_2) * 0.45 if max(num_1, num_2) > 0 else 6))
    fig_width = 10
    fontsize = 9 if max(num_1, num_2) < 20 else 8

    plt.style.use('default') # Ensure default style for consistency
    fig, ax = plt.subplots(figsize=(fig_width, fig_height), dpi=120)

    plot_text_color = 'black'
    plot_bg_color = 'white'

    max_items_plot = max(num_1, num_2)
    y_1 = np.linspace(max_items_plot -1 , 0, num_1) if num_1 > 0 else np.array([])
    y_2 = np.linspace(max_items_plot -1, 0, num_2) if num_2 > 0 else np.array([])

    x_pos_1 = 0.5
    x_pos_2 = 4.5
    line_x_start = x_pos_1 + 0.6
    line_x_end = x_pos_2 - 0.6

    for i, item1 in enumerate(items_1):
        if y_1.size > i:
            ax.text(x_pos_1, y_1[i], f"{prefix_1}-{i+1}: {truncar_texto(item1, MAX_TEXT_LEN_VIZ-10)}",
                                ha='right', va='center', fontsize=fontsize, fontweight='bold', color=plot_text_color)
    for j, item2 in enumerate(items_2):
        if y_2.size > j:
            ax.text(x_pos_2, y_2[j], f"{prefix_2}-{j+1}: {truncar_texto(item2, MAX_TEXT_LEN_VIZ-10)}",
                                ha='left', va='center', fontsize=fontsize, color=plot_text_color)

    color_map = plt.get_cmap(cmap)
    norm = mcolors.Normalize(vmin=umbral_conexion, vmax=1.0)

    conexiones_dibujadas = 0
    for i in range(num_1):
        for j in range(num_2):
            sim = matriz[i, j]
            if sim >= umbral_conexion:
                if y_1.size > i and y_2.size > j:
                    color_line = color_map(norm(sim))
                    ancho_linea = 0.5 + (sim - umbral_conexion) * 4.0 / (1.0 - umbral_conexion + 1e-6)
                    ax.plot([line_x_start, line_x_end], [y_1[i], y_2[j]],
                            color=color_line, alpha=0.7, linewidth=max(0.5, ancho_linea), zorder=1)
                    conexiones_dibujadas +=1

    if conexiones_dibujadas > 0:
        from matplotlib.cm import ScalarMappable
        sm = ScalarMappable(cmap=color_map, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, orientation='horizontal', pad=0.08, fraction=0.05, aspect=30, ax=ax)
        cbar.set_label(f"Similitud ≥ {umbral_conexion:.2f}", size=fontsize, color=plot_text_color)
        cbar.ax.tick_params(axis='y', colors=plot_text_color)
        cbar.outline.set_edgecolor(plot_text_color)

    # MEJORA: Se convierte explícitamente a string para evitar errores sutiles de renderizado.
    ax.set_title(f"{str(titulo)} (Similitud ≥ {umbral_conexion:.2f})", fontsize=fontsize + 3, pad=20, color=plot_text_color)
    ax.text(x_pos_1, max_items_plot + 0.5 if max_items_plot > 0 else 1, label_1, ha='center', va='bottom', fontsize=fontsize + 1, fontweight='bold', color=plot_text_color)
    ax.text(x_pos_2, max_items_plot + 0.5 if max_items_plot > 0 else 1, label_2, ha='center', va='bottom', fontsize=fontsize + 1, fontweight='bold', color=plot_text_color)

    ax.set_xlim(0, x_pos_2 + 1)
    ax.set_ylim(-1, max_items_plot + 1.5)
    ax.axis('off')
    fig.patch.set_facecolor(plot_bg_color)

    buf = io.BytesIO()
    try:
        fig.savefig(buf, format='png', bbox_inches='tight', facecolor=fig.get_facecolor())
        buf.seek(0)
        plt.close(fig)
    except Exception as e:
        logging.error(f"Error generar diagrama conexiones: {e}", exc_info=True)
        if 'fig' in locals() and fig:
            plt.close(fig)
        return None
    return buf

# --- Generación de HTML para Resumen en Colab (Dark Theme) ---

def generar_html_resultado_comparacion(label: str, result: str, success: bool | None = None) -> str:
    """Genera un bloque HTML para un criterio: etiqueta, mensaje, icono y color."""
    if success is None: # Neutral or informational
        color = AppColors.TEXT_DARK.value.hexval()
        icon = "<i class='fa fa-info-circle' style='color:#4fc3f7;'></i> "
    elif success:
        color = AppColors.SUCCESS.value.hexval()
        icon = "<i class='fa fa-check' style='color:#66bb6a;'></i> "
    else:
        color = AppColors.DANGER.value.hexval()
        icon = "<i class='fa fa-times' style='color:#e57373;'></i> "
    return f"<p style='margin:2px 0; font-size:0.95em;'><b>{label}:</b> <span style='color:{color};'>{icon}{result}</span></p>"


def generar_html_similitud_aspecto_informativo(label: str, metrics: dict, tiene_datos_validos_ambos: bool, error_msg: str | None = None, ref_model_desc: str = "N/A", info_extra: str | None = None) -> str:
    """Genera HTML de similitud para un aspecto (Estudio, Experiencia) de forma informativa."""
    if error_msg:
        return (
            f"<div style='border-left:3px solid {AppColors.DANGER.value.hexval()}; "
            f"padding-left:8px; margin:5px 0;'><p><b>{label}:</b> "
            f"<span style='color:{AppColors.DANGER.value.hexval()};'><i class='fa fa-times'></i> Error: {error_msg[:100]}... (Ref: {ref_model_desc})</span></p></div>"
        )
    elif not tiene_datos_validos_ambos:
        return (
            f"<p style='margin:2px 0; font-size:0.9em;'><b>{label}:</b> "
            f"<span style='color:{AppColors.TEXT_LIGHT.value.hexval()};'><i>No aplica (sin datos válidos en ambos empleos). (Ref: {ref_model_desc})</i></span></p>"
        )
    elif not metrics:
        msg = "No se pudo comparar (datos válidos en un solo empleo o no procesables)."
        if info_extra:
            msg = info_extra
        return (
            f"<div style='border-left:3px solid {AppColors.WARNING.value.hexval()}; "
            f"padding-left:8px; margin:5px 0;'><p><b>{label}:</b> "
            f"<span style='color:{AppColors.WARNING.value.hexval()};'><i class='fa fa-exclamation-triangle'></i> "
            f"{msg} (Ref: {ref_model_desc})</span></p></div>"
        )

    cob1 = metrics.get('cobertura_prom_max_1', 0.0)
    nivel, color_nivel = obtener_nivel_similitud(cob1)

    html = (
        f"<div style='border-left:3px solid {color_nivel}; padding-left:8px; margin:5px 0;'>"
        f"<p style='margin:2px 0; font-weight:bold;'>{label} (Análisis Informativo de Similitud):</p>"
        f"<ul style='margin:0; padding-left:20px; font-size:0.9em;'>"
    )
    if info_extra:
         html += f"<li><i>Análisis Jerárquico: {info_extra}</i></li>"
    html += (
        f"<li>Cobertura Promedio (E1 por E2): <b>{cob1:.2%}</b> (Nivel: <span style='color:{color_nivel};'>{nivel}</span>) (Ref: {ref_model_desc})</li>"
        f"<li><i>Nota: Este análisis es de referencia. La verificación de cumplimiento de requisitos es manual.</i></li>"
        f"</ul></div>"
    )
    return html


def generar_html_similitud_criterio_decision(
    label: str,
    metrics: dict,
    umbral_decision: float,
    pasa_criterio: bool,
    tiene_datos_validos_ambos: bool,
    error_msg: str | None = None,
    ref_model_desc: str = "N/A"
) -> str:
    """Genera HTML para un criterio de decisión basado en similitud (Estudio, Competencias)."""
    if error_msg:
        return generar_html_resultado_comparacion(label, f"Error: {error_msg[:80]}... (Ref: {ref_model_desc})", False)

    if not tiene_datos_validos_ambos:
        return generar_html_resultado_comparacion(label, f"No aplica (sin datos válidos en ambos). (Ref: {ref_model_desc})", None)

    if not metrics:
        return generar_html_resultado_comparacion(label, f"No comparable (datos válidos en un solo lado o no procesables). (Ref: {ref_model_desc})", False)

    cob1 = metrics.get('cobertura_prom_max_1', 0.0)
    nivel, _ = obtener_nivel_similitud(cob1)
    detalle_msg = f"Cob. Promedio (E1 x E2): {cob1:.2%} - Nivel: {nivel} (Ref: {ref_model_desc})"
    return generar_html_resultado_comparacion(f"{label} (Similitud ≥ {umbral_decision:.0%})", detalle_msg, pasa_criterio)


def mostrar_resumen_equivalencia_html(results: dict[str, object]) -> str:
    """Genera el bloque HTML de resumen ejecutivo de equivalencia (Dark Theme)."""
    params = results.get('params', {})
    eq_res = results.get('equivalencia_results', {})

    # NUEVO: Leer si los requisitos se incluyen en la decisión
    incluir_reqs_en_decision = params.get('incluir_requisitos_en_decision', True)

    simil_func = results.get('similitud_funciones', {})
    simil_edu = results.get('similitud_educacion', {})
    simil_exp = results.get('similitud_experiencia', {})
    simil_cl = results.get('similitud_comp_lab', {})
    simil_cc = results.get('similitud_comp_comp', {})

    if not eq_res:
        return "<p>No hay resultados de equivalencia para mostrar.</p>"

    base_denom = params.get('Denominación', 'N/A')
    base_codigo = params.get('Código','N/A')
    base_opec = params.get(ID_COLUMN_NAME, 'N/A')
    base_grado = params.get('Grado','N/A')

    target_denom = eq_res.get('target_Denominacion', 'N/A')
    target_codigo = eq_res.get('target_Codigo', 'N/A')
    target_opec = eq_res.get('target_OPEC', 'N/A')

    nivel_res = eq_res.get('nivel', {})
    salario_res = eq_res.get('salario', {})

    ref_model_id = results.get('reference_model_id')
    ref_model_desc = "N/A"
    if ref_model_id and results.get('results_per_model'):
        ref_model_desc = results.get('results_per_model',{}).get(ref_model_id, {}).get('model_desc', ref_model_id).split(' (')[0]

    metrics_func = simil_func.get('metricas', {})
    metrics_edu = simil_edu.get('metricas', {})
    metrics_exp = simil_exp.get('metricas', {})
    metrics_cl = simil_cl.get('metricas', {})
    metrics_cc = simil_cc.get('metricas', {})

    equivalente = eq_res.get('es_equivalente', False)
    color_final = AppColors.SUCCESS.value.hexval() if equivalente else AppColors.DANGER.value.hexval()
    texto_final = "SÍ SON EQUIVALENTES (AUTOMATIZADO)" if equivalente else "NO SON EQUIVALENTES (AUTOMATIZADO)"
    razones = eq_res.get('razones_no_equivalencia', [])
    equivalence_score_display = eq_res.get('equivalence_score', 0.0)

    html = (
        f"<div style='border:1px solid {color_final}; border-left:7px solid {color_final}; "
        f"padding:15px; margin:10px 0; background-color:{AppColors.ACCENT_BACKGROUND.value.hexval()}; "
        f"border-radius:5px; box-shadow:2px 2px 5px rgba(0,0,0,0.5); color:{AppColors.TEXT_DARK.value.hexval()};'>"
        f"<h3 style='margin-top:0; text-align:center; color:{AppColors.PRIMARY.value.hexval()};"
        f"font-size:1.2em; border-bottom:2px solid {AppColors.PRIMARY.value.hexval()}; padding-bottom:5px;'>"
        f"INFORME EJECUTIVO DE EQUIVALENCIA</h3>"
        f"<p><b>Empleo Base:</b> {base_denom} (OPEC: {base_opec}, Código: {base_codigo}, Grado: {base_grado})</p>"
        f"<p><b>Empleo Destino:</b> {target_denom} (OPEC: {target_opec}, Código: {target_codigo})</p>"
        f"<p><b>Profesión Aspirante/Servidor:</b> {params.get('Profesion Aspirante', 'No especificada')}</p>"
        f"<p><b>Tipo de Uso del Estudio:</b> {params.get('Tipo de Uso', 'No especificado')}"
        f"{' (' + params.get('Tipo de Uso Otro', '') + ')' if params.get('Tipo de Uso Otro') else ''}</p>"
        f"<hr style='border-top:1px dashed {AppColors.ACCENT_BORDER.value.hexval()}; margin:10px 0;'/>"
        f"<h4 style='color:{AppColors.SECONDARY.value.hexval()}; margin-bottom:5px;'>Resultados de la Comparación Automatizada:</h4>"
    )

    # Criterios base (siempre decisivos)
    html += generar_html_resultado_comparacion("1. Nivel Jerárquico", nivel_res.get('mensaje','Error'), nivel_res.get('pasa', False))
    html += generar_html_resultado_comparacion("2. Salario / Grado", salario_res.get('mensaje','Error'), salario_res.get('pasa', False))
    html += generar_html_similitud_criterio_decision(
        "3. Funciones", metrics_func, params.get('umbral_decision_funciones', 0.6),
        eq_res.get('pasa_funciones', False),
        _is_valid_data(params.get('Funciones Base','')) and _is_valid_data(eq_res.get('raw_dest', {}).get('Funciones','')),
        simil_func.get('error'), ref_model_desc
    )

    html += f"<hr style='border-top:1px dashed {AppColors.ACCENT_BORDER.value.hexval()}; margin:10px 0;'/>"

    if incluir_reqs_en_decision:
        html += f"<h4 style='color:{AppColors.SECONDARY.value.hexval()}; margin-bottom:5px;'>Análisis de Requisitos (Incluidos en Decisión):</h4>"
        # Requisitos Estudio
        html += generar_html_similitud_criterio_decision(
            "4. Requisitos de Estudio", metrics_edu, params.get('umbral_decision_requisitos_estudio', 0.7),
            eq_res.get('pasa_educacion_decision', False),
            _is_valid_data(params.get('Requisitos Estudio Base','')) and _is_valid_data(eq_res.get('raw_dest', {}).get('Requisitos Estudio','')),
            simil_edu.get('error'), ref_model_desc
        )
        # Competencias
        umbral_comp = params.get('umbral_decision_requisitos_comp', DEFAULT_UMBRAL_DECISION_REQUISITOS_COMP)
        html += generar_html_similitud_criterio_decision(
            "6. Competencias Laborales", metrics_cl, umbral_comp,
            eq_res.get('pasa_comp_lab', False),
            _is_valid_data(params.get('Competencias Laborales Base','')) and _is_valid_data(eq_res.get('raw_dest', {}).get('Competencias Laborales','')),
            simil_cl.get('error'), ref_model_desc
        )
        html += generar_html_similitud_criterio_decision(
            "7. Competencias Comportamentales", metrics_cc, umbral_comp,
            eq_res.get('pasa_comp_comp', False),
            _is_valid_data(params.get('Competencias Comportamentales Base','')) and _is_valid_data(eq_res.get('raw_dest', {}).get('Competencias Comportamentales','')),
            simil_cc.get('error'), ref_model_desc
        )
    else:
        html += f"<h4 style='color:{AppColors.TEXT_LIGHT.value.hexval()}; margin-bottom:5px;'>Análisis de Requisitos (Modo Informativo - No afecta decisión):</h4>"
        # Requisitos Estudio (Informativo)
        html += generar_html_similitud_aspecto_informativo("4. Requisitos de Estudio", metrics_edu, _is_valid_data(params.get('Requisitos Estudio Base','')) and _is_valid_data(eq_res.get('raw_dest', {}).get('Requisitos Estudio','')), simil_edu.get('error'), ref_model_desc)
        # Competencias (Informativo)
        html += generar_html_similitud_aspecto_informativo("6. Competencias Laborales", metrics_cl, _is_valid_data(params.get('Competencias Laborales Base','')) and _is_valid_data(eq_res.get('raw_dest', {}).get('Competencias Laborales','')), simil_cl.get('error'), ref_model_desc)
        html += generar_html_similitud_aspecto_informativo("7. Competencias Comportamentales", metrics_cc, _is_valid_data(params.get('Competencias Comportamentales Base','')) and _is_valid_data(eq_res.get('raw_dest', {}).get('Competencias Comportamentales','')), simil_cc.get('error'), ref_model_desc)

    # Requisitos Experiencia (Siempre informativo)
    html += generar_html_similitud_aspecto_informativo(
        "5. Requisitos de Experiencia", metrics_exp,
        _is_valid_data(params.get('Requisitos Experiencia Base','')) and _is_valid_data(eq_res.get('raw_dest', {}).get('Requisitos Experiencia','')),
        simil_exp.get('error'), ref_model_desc, info_extra=simil_exp.get('jerarquia_msg')
    )

    html += (
        f"<hr style='border-top:1px solid {AppColors.ACCENT_BORDER.value.hexval()}; margin:15px 0;'/>"
        f"<div style='text-align:center; margin-top:10px; padding:10px; border:2px solid {color_final}; "
        f"background-color:{AppColors.ACCENT_BACKGROUND.value.hexval()}; border-radius:4px;'>"
        f"<h4 style='margin:0; color:{color_final}; font-size:1.1em;'>DECISIÓN AUTOMATIZADA DE EQUIVALENCIA</h4>"
        f"<p style='margin:5px 0 0 0; font-size:1.2em; font-weight:bold;'>{texto_final}</p>"
        f"<p style='margin:5px 0 0 0; font-size:1.1em; font-weight:bold; color:{color_final}'>Puntuación de Equivalencia (ponderada): {equivalence_score_display:.2%}</p>"
    )
    if not incluir_reqs_en_decision:
        html += f"<p style='font-size:0.8em; color:{AppColors.WARNING.value.hexval()}; margin-top:5px;'><i>(Decisión basada únicamente en Nivel, Salario y Funciones)</i></p>"

    if not equivalente and razones:
        html += "<p style='margin:5px 0 0 0; font-size:0.9em;'><i>Motivos principales (automatizados):</i></p>"
        html += "<ul style='text-align:left; margin:0 auto; padding-left:30px; font-size:0.9em; max-width: 90%;'>"
        for r in razones:
            html += f"<li>{r}</li>"
        html += "</ul>"
    html += (
        f"</div>"
        f"<p style='font-size:0.9em; color:{AppColors.WARNING.value.hexval()}; margin-top:10px; text-align:center;'>"
        f"<b>Importante:</b> La verificación de requisitos de Estudio y Experiencia es un proceso manual que debe ser realizado por el analista y documentado en el informe PDF.</p>"
        f"<p style='font-size:0.9em; color:{AppColors.TEXT_LIGHT.value.hexval()}; margin-top:5px; text-align:right;'>"
        f"<i>Análisis de apoyo (Ref: {ref_model_desc}). Requiere validación experta final.</i></p>"
        f"</div>"
    )
    return html

def generar_tabla_comparacion_global_html(results: dict[str, object]) -> str:
    """Genera una tabla HTML comparativa global para todos los modelos y criterios (Dark Theme)."""
    params = results.get('params', {})
    eq_res = results.get('equivalencia_results', {})
    per_model_results_for_target = results.get('results_per_model', {})
    ref_id_for_target = results.get('reference_model_id')

    incluir_reqs = params.get('incluir_requisitos_en_decision', True)

    if not per_model_results_for_target:
        return "<p><i>No hay resultados por modelo para generar tabla global para este destino.</i></p>"

    model_ids_used = list(per_model_results_for_target.keys())
    num_models = len(model_ids_used)

    header_html = "<tr><th style='border:1px solid #616161; padding:6px; background-color:#424242; color:white; width:20%;'>Criterio</th>"
    for mid in model_ids_used:
        desc = per_model_results_for_target.get(mid, {}).get('model_desc', mid).split(' (')[0]
        is_ref = "<span style='color:#FFD700;'>(REF)</span>" if mid == ref_id_for_target else ""
        header_html += f"<th style='border:1px solid #616161; padding:6px; background-color:#424242; color:white; text-align:center;'>{desc} {is_ref}</th>"
    header_html += "</tr>"

    rows_html = ""

    def format_cell_global(pasa: bool | None, metric: float | None = None, text_override: str | None = None, error: bool = False, informativo: bool = False) -> str:
        if error:
            return f"<span style='color:{AppColors.DANGER.value.hexval()};'><i class='fa fa-times'></i> Error</span>"

        current_color = AppColors.TEXT_DARK.value.hexval()
        current_icon = ""
        if pasa is not None:
            if informativo:
                    current_color = AppColors.INFO.value.hexval()
                    current_icon = "<i class='fa fa-info-circle'></i> "
            else:
                    current_color = AppColors.SUCCESS.value.hexval() if pasa else AppColors.DANGER.value.hexval()
                    current_icon = "<i class='fa fa-check'></i> " if pasa else "<i class='fa fa-times'></i> "

        if text_override:
            return f"<span style='color:{current_color};'>{current_icon}{text_override}</span>"

        if pasa is None: # N/A
            return f"<span style='color:{AppColors.TEXT_LIGHT.value.hexval()};'><i>N/A</i></span>"

        metric_str = f" ({metric:.1%})" if metric is not None else ""
        pf_text = "Pasa" if pasa else "No Pasa"
        if informativo:
            pf_text = f"Sim: {metric:.1%}" if metric is not None else "N/A"

        return f"<span style='color:{current_color};'>{current_icon}{pf_text}{metric_str if not informativo or (informativo and metric is not None) else ''}</span>"


    nivel_res_target = eq_res.get('nivel', {})
    pasa_nivel_target = nivel_res_target.get('pasa')
    nivel_msg_target = nivel_res_target.get('mensaje','').split('.')[0]
    rows_html += (
        "<tr><td style='border:1px solid #616161; padding:6px;'><b>1. Nivel Jerárquico</b></td>"
        f"<td colspan='{num_models}' style='border:1px solid #616161; padding:6px; text-align:center;'>{format_cell_global(pasa_nivel_target, text_override=nivel_msg_target)}</td></tr>"
    )

    salario_res_target = eq_res.get('salario', {})
    pasa_sal_target = salario_res_target.get('pasa')
    sal_msg_target = salario_res_target.get('mensaje', '').split(': ')[-1].split('.')[0]
    rows_html += (
        "<tr><td style='border:1px solid #616161; padding:6px;'><b>2. Salario / Grado</b></td>"
        f"<td colspan='{num_models}' style='border:1px solid #616161; padding:6px; text-align:center;'>{format_cell_global(pasa_sal_target, text_override=sal_msg_target)}</td></tr>"
    )

    criterios_similitud = [
        ('3. Funciones', 'similitud_funciones', 'umbral_decision_funciones', 'Funciones Base', 'Funciones', False),
        (f'4. Requisitos de Estudio{" (Info)" if not incluir_reqs else ""}', 'similitud_educacion', 'umbral_decision_requisitos_estudio', 'Requisitos Estudio Base', 'Requisitos Estudio', not incluir_reqs),
        ('5. Requisitos de Experiencia (Info)', 'similitud_experiencia', 'umbral_decision_requisitos_comp', 'Requisitos Experiencia Base', 'Requisitos Experiencia', True),
        (f'6. Competencias Laborales{" (Info)" if not incluir_reqs else ""}', 'similitud_comp_lab', 'umbral_decision_requisitos_comp', 'Competencias Laborales Base', 'Competencias Laborales', not incluir_reqs),
        (f'7. Competencias Comportamentales{" (Info)" if not incluir_reqs else ""}', 'similitud_comp_comp', 'umbral_decision_requisitos_comp', 'Competencias Comportamentales Base', 'Competencias Comportamentales', not incluir_reqs)
    ]

    for label, key_sim_results, umbral_key_params, raw1_key_params, raw2_key_params, es_informativo in criterios_similitud:
        rows_html += f"<tr><td style='border:1px solid #616161; padding:6px;'><b>{label}</b></td>"
        umbral_valor = params.get(umbral_key_params, 0.65)

        tiene_datos_base_validos = _is_valid_data(str(params.get(raw1_key_params, "")))
        raw_dest_data_for_criterion = eq_res.get('raw_dest', pd.Series(dtype=object))
        tiene_datos_destino_validos = _is_valid_data(str(raw_dest_data_for_criterion.get(raw2_key_params,"")))

        for mid in model_ids_used:
            model_specific_result = per_model_results_for_target.get(mid, {})
            sim_result_for_model_criterion = model_specific_result.get(key_sim_results, {})

            cell_content = ""
            if model_specific_result.get('error_general'):
                cell_content = format_cell_global(None, error=True, informativo=es_informativo)
            elif sim_result_for_model_criterion.get('error'):
                cell_content = format_cell_global(None, error=True, informativo=es_informativo)
            elif not (tiene_datos_base_validos or tiene_datos_destino_validos):
                cell_content = format_cell_global(None, informativo=es_informativo)
            elif not (tiene_datos_base_validos and tiene_datos_destino_validos):
                cell_content = format_cell_global(False, text_override="No comparable", informativo=es_informativo)
            elif not sim_result_for_model_criterion.get('metricas'):
                # Puede haber mensaje de jerarquía en experiencia
                info_msg = sim_result_for_model_criterion.get('jerarquia_msg')
                if info_msg:
                           cell_content = format_cell_global(False, text_override=info_msg, informativo=True)
                else:
                           cell_content = format_cell_global(False, text_override="Error Procesando", informativo=es_informativo)
            else:
                metrics_criterion = sim_result_for_model_criterion.get('metricas', {})
                metric_val = metrics_criterion.get('cobertura_prom_max_1', 0.0)
                pasa_criterion = metric_val >= umbral_valor
                if es_informativo:
                    cell_content = format_cell_global(True, metric=metric_val, informativo=True)
                else:
                    cell_content = format_cell_global(pasa_criterion, metric=metric_val, informativo=False)

            rows_html += f"<td style='border:1px solid #616161; padding:6px; text-align:center;'>{cell_content}</td>"
        rows_html += "</tr>"

    final_pasa_target = eq_res.get('es_equivalente', False)
    final_html_target = format_cell_global(final_pasa_target, text_override="EQUIVALENTE (Auto)" if final_pasa_target else "NO EQUIVALENTE (Auto)")

    rows_html += (
        "<tr><td style='border:1px solid #616161; padding:6px; background-color:#424242; color:white;'>"
        "<b>DECISIÓN AUTOMATIZADA</b><br/><small>(Basada en Modelo REF. "
        f"{'Requisitos NO incluidos' if not incluir_reqs else 'Requisitos incluidos'})</small></td>"
    )

    for i, mid in enumerate(model_ids_used):
        style_extra = ""
        content_cell = "<span style='color:grey;'><i>(ver Ref.)</i></span>"
        if mid == ref_id_for_target:
            style_extra = f"border:2px solid {AppColors.SUCCESS.value.hexval() if final_pasa_target else AppColors.DANGER.value.hexval()};"
            content_cell = final_html_target

        rows_html += f"<td style='border:1px solid #616161; padding:10px; text-align:center; {style_extra}'>{content_cell}</td>"
    rows_html += "</tr>"

    table_html = (
        "<div style='margin-top:20px;'>"
        f"<h4 style='color:{AppColors.SECONDARY.value.hexval()}; margin-bottom:5px;'>"
        "Tabla Comparativa Global por Modelo y Criterio (para este Destino)</h4>"
        "<table style='border-collapse:collapse; width:100%; table-layout:fixed; font-size:0.85em;'>"
        f"<thead>{header_html}</thead><tbody>{rows_html}</tbody></table>"
        f"<p style='font-size:0.8em; color:{AppColors.TEXT_LIGHT.value.hexval()}; margin-top:5px;'>"
        "<i>N/A: No aplica (sin datos válidos en ambos empleos para el criterio). No comparable: Datos válidos presentes pero no analizables. "
        "Similitud calculada como Cobertura Promedio Máxima (E1 x E2).</i></p>"
        "</div>"
    )
    return table_html

# --- FUNCION NUEVA: Mostrar detalles de similitud en HTML ---
def mostrar_detalles_similitud_html(
    titulo_base_viz: str,
    items1_viz: list[str],
    items2_reord_viz: list[str],
    matriz_viz: np.ndarray,
    prefix1: str,
    prefix2: str,
    top_n: int,
    umbral_cob_bin: float,
    ref_model_desc: str,
    info_extra_header: str | None = None
) -> str:
    """Genera el bloque HTML de detalles de similitud para un aspecto (Dark Theme)."""

    # Check if there is anything to display
    if (not items1_viz and not items2_reord_viz):
        return (f"<div style='border:1px solid {AppColors.ACCENT_BORDER.value.hexval()}; padding:10px; margin-top:15px; border-radius:5px;'>"
                f"<h4 style='color:{AppColors.PRIMARY.value.hexval()}; margin-top:0;'>{titulo_base_viz} (Ref: {ref_model_desc})</h4>"
                f"<p style='color:{AppColors.WARNING.value.hexval()};'><i>No se encontraron ítems válidos para procesar en este aspecto para ninguno de los empleos.</i></p></div>")

    # Metrics calculation for the header
    metricas_viz = calcular_metricas_agregadas(matriz_viz)
    cob_prom_max_1 = metricas_viz.get('cobertura_prom_max_1', 0.0)
    cob_bin1_viz = calcular_cobertura_binaria(matriz_viz, umbral_cob_bin, axis=1)
    nivel_sim_viz, color_nivel_viz = obtener_nivel_similitud(cob_prom_max_1)

    html = (
        f"<div style='border:1px solid {AppColors.ACCENT_BORDER.value.hexval()}; padding:10px; margin-top:15px; border-radius:5px;'>"
        f"<h4 style='color:{AppColors.PRIMARY.value.hexval()}; margin-top:0;'>{titulo_base_viz} (Ref: {ref_model_desc})</h4>"
    )
    if info_extra_header:
        html += f"<p style='font-size:0.9em; color:{AppColors.INFO.value.hexval()}; margin-bottom:10px;'><i><b>Análisis Jerárquico:</b> {info_extra_header}</i></p>"
    html += (
        f"<div style='display:flex; flex-wrap:wrap; justify-content:space-around; font-size:0.9em; margin-bottom:10px; background-color:{AppColors.ACCENT_BACKGROUND.value.hexval()}; padding:5px; border-radius:3px;'>"
        f"<span><b>Items Base:</b> {len(items1_viz)}</span>"
        f"<span><b>Items Destino:</b> {len(items2_reord_viz)}</span>"
        f"<span><b>Cob. Prom. Máx (Base x Dest.):</b> <span style='color:{color_nivel_viz}; font-weight:bold;'>{cob_prom_max_1:.2%}</span> ({nivel_sim_viz})</span>"
        f"<span><b>Cob. Binaria (≥{umbral_cob_bin:.0%}):</b> {cob_bin1_viz:.2%}</span>"
        f"</div>"
    )

    if matriz_viz.size == 0:
        html += f"<p style='color:{AppColors.WARNING.value.hexval()};'><i>No se pudo generar la matriz de similitud (e.g., solo un empleo tenía ítems o la jerarquía no es válida).</i></p></div>"
        return html

    # Build the details table
    html += "<table style='width:100%; border-collapse:collapse; table-layout:fixed; font-size:0.85em;'>"
    html += (
        "<thead><tr style='background-color:#424242;'>"
        f"<th style='width:50%; padding:5px; border:1px solid #616161;'>Item Empleo Base ({prefix1})</th>"
        f"<th style='width:50%; padding:5px; border:1px solid #616161;'>Mejores Coincidencias Empleo Destino ({prefix2}) (Similitud)</th>"
        "</tr></thead><tbody>"
    )

    for i, item1 in enumerate(items1_viz):
        if not items2_reord_viz:
            top_matches_html = "<i>(No hay ítems en Empleo Destino para comparar)</i>"
        else:
            sim_scores = matriz_viz[i, :]
            top_indices = np.argsort(sim_scores)[::-1][:top_n]

            top_matches = []
            for j in top_indices:
                score = sim_scores[j]
                item2 = items2_reord_viz[j]
                match_color = obtener_nivel_similitud(score)[1]
                top_matches.append(
                    f"<div style='border-left:3px solid {match_color}; padding-left:5px; margin-bottom:3px;'>"
                    f"<b>({score:.2f})</b> {truncar_texto(item2, 100)}"
                    "</div>"
                )
            top_matches_html = "".join(top_matches) if top_matches else "<i>(Sin coincidencias significativas)</i>"

        html += (
            f"<tr style='background-color:#2a2a2a;'>"
            f"<td style='padding:5px; border:1px solid #616161; vertical-align:top;'><b>{prefix1}-{i+1}:</b> {truncar_texto(item1, 150)}</td>"
            f"<td style='padding:5px; border:1px solid #616161; vertical-align:top;'>{top_matches_html}</td>"
            "</tr>"
        )

    html += "</tbody></table></div>"
    return html

# --- Funciones de Comparaciones Directas ---

def comparar_niveles(n1: str, n2: str) -> dict[str, object]:
    """Compara nivel jerárquico: debe ser idéntico para pasar."""
    n1_norm = normalizar_texto_func(str(n1 or "")).strip()
    n2_norm = normalizar_texto_func(str(n2 or "")).strip()

    if not n1_norm or not n2_norm:
        return {"pasa": False, "mensaje": f"Niveles no especificados o inválidos: Base='{n1}', Destino='{n2}'. No Pasa."}

    if n1_norm == n2_norm:
        return {"pasa": True, "mensaje": f"Ambos son Nivel '{n1}'. Pasa."}
    else:
        return {"pasa": False, "mensaje": f"Niveles diferentes: Base='{n1}', Destino='{n2}'. No Pasa."}

def comparar_salarios(
    s1: float, s2: float,
    g1: int, g2: int,
    orden1: str, nat1: str | None,
    orden2: str, nat2: str | None
) -> dict[str, object]:
    """Compara salario y grado según reglas (2 grados o 10% ABM)."""
    try:
        s1 = float(s1)
        s2 = float(s2)
        g1 = int(g1) if pd.notna(g1) else -1
        g2 = int(g2) if pd.notna(g2) else -1
    except (ValueError, TypeError):
        return {"pasa": False, "mensaje": "Salarios o grados con formato inválido. No Pasa."}

    if s1 <= 0 or s2 <= 0:
        return {"pasa": False, "mensaje": f"Salarios inválidos (Base: {s1:,.0f}, Destino: {s2:,.0f}). Deben ser > 0. No Pasa."}

    # VALIDACIÓN CRÍTICA: Los grados deben ser exactamente iguales
    if g1 != -1 and g2 != -1 and g1 != g2:
        return {
            "pasa": False,
            "mensaje": f"Grados diferentes: Base(G{g1}) ≠ Destino(G{g2}). Se requiere MISMO GRADO exacto. No Pasa."
        }

    ENTIDADES_NACIONAL_REGLA_GRADOS = [
        "ministerio", "departamento administrativo", "superintendencia",
        "unidad administrativa especial", "establecimiento publico",
        "corporacion autonoma regional y de desarrollo sostenible",
        "empresa social del estado (ese)",
        "empresa industrial y comercial del estado (eice)",
        "sociedad de economia mixta (regimen eice)",
        "otra institucion publica ejecutiva nacional",
        "entidad en liquidacion nacional"
    ]

    orden1_norm = normalizar_texto_func(str(orden1 or ""))
    orden2_norm = normalizar_texto_func(str(orden2 or ""))
    nat1_norm = normalizar_texto_func(str(nat1 or ""))
    nat2_norm = normalizar_texto_func(str(nat2 or ""))

    aplicar_grados = False
    if orden1_norm == "nacional" and orden2_norm == "nacional" and \
       nat1_norm in ENTIDADES_NACIONAL_REGLA_GRADOS and \
       nat2_norm in ENTIDADES_NACIONAL_REGLA_GRADOS and \
       g1 != -1 and g2 != -1:
        aplicar_grados = True

    if aplicar_grados:
        pasa_sal_min = (s2 >= s1)
        pasa_g_igual = (g2 == g1)  # Cambio: debe ser exactamente igual

        if pasa_sal_min and pasa_g_igual:
            return {
                "pasa": True,
                "mensaje": f"Regla Mismo Grado (Nacional): E2(G{g2}, ${s2:,.0f}) vs E1(G{g1}, ${s1:,.0f}). Salario OK, Grado OK. Pasa."
            }
        else:
            razones_falla = []
            if not pasa_sal_min: razones_falla.append(f"Salario E2(${s2:,.0f}) < E1(${s1:,.0f})")
            if not pasa_g_igual: razones_falla.append(f"Grado E2({g2}) ≠ E1({g1}) - Se requiere mismo grado exacto")
            return {
                "pasa": False,
                "mensaje": f"Regla Mismo Grado (Nacional): No Pasa. Fallos: {'; '.join(razones_falla)}."
            }
    else:
        if s2 < s1:
            return {"pasa": False, "mensaje": f"Regla 10% ABM: Salario E2(${s2:,.0f}) < E1(${s1:,.0f}). No Pasa."}

        limite_superior_salario = s1 * (1 + SALARY_DIFF_PERCENTAGE_THRESHOLD)
        diferencia_relativa = ((s2 - s1) / s1) if s1 > 0 else 0

        if s2 <= limite_superior_salario:
            return {
                "pasa": True,
                "mensaje": f"Regla 10% ABM: E2(${s2:,.0f}) vs E1(${s1:,.0f}). Diferencia ({diferencia_relativa:.1%}) ≤ {SALARY_DIFF_PERCENTAGE_THRESHOLD:.0%}. Pasa."
            }
        else:
            return {
                "pasa": False,
                "mensaje": f"Regla 10% ABM: E2(${s2:,.0f}) vs E1(${s1:,.0f}). Diferencia ({diferencia_relativa:.1%}) > {SALARY_DIFF_PERCENTAGE_THRESHOLD:.0%}. No Pasa."
            }

def calcular_similitud_aspecto(
    texto1: str,
    texto2: str,
    aspecto: str, # e.g., "Funciones", "Educacion", etc.
    params: dict[str, object],
    nlp_model: spacy.language.Language | None,
    sbert_model: SentenceTransformer,
    device: str
) -> dict[str, object]:
    """Calcula similitud para un aspecto genérico (Funciones, Educación, Competencias)."""
    resultados: dict[str, object] = {
        'items1_proc': [], 'items2_proc': [],
        'matriz': np.array([]), 'metricas': {},
        'items2_proc_reordenados': [],
        'heatmap_buffer': None, 'conex_buffer': None,
        'error': None
    }
    try:
        items1 = procesar_bloque(
            texto1, aspecto,
            params['eliminar_dups'], params['normalizar'], params['lematizar'],
            nlp_model, params['min_words']
        )
        items2 = procesar_bloque(
            texto2, aspecto,
            params['eliminar_dups'], params['normalizar'], params['lematizar'],
            nlp_model, params['min_words']
        )
        resultados['items1_proc'] = items1
        resultados['items2_proc'] = items2

        if not items1 or not items2:
            msg = "No hay items válidos para "
            if not items1 and not items2: msg += "ningún empleo."
            elif not items1: msg += "el empleo base."
            else: msg += "el empleo destino."
            resultados['error'] = f"Datos insuficientes en '{aspecto}'. {msg}"
            logging.warning(resultados['error'])
            return resultados

        matriz_sim_original = calcular_matriz_similitud_combinada(
            items1, items2, sbert_model, device,
            params['usar_jaccard'], params['peso_cos'], params['peso_jacc']
        )
        if matriz_sim_original.shape != (len(items1), len(items2)):
            raise ValueError(f"Matriz de similitud inválida para '{aspecto}'.")

        matriz_reordenada, items2_reordenados, _ = reordenar_columnas_por_similitud(matriz_sim_original, items2)
        metricas = calcular_metricas_agregadas(matriz_reordenada)

        resultados.update({
            'matriz': matriz_reordenada,
            'metricas': metricas,
            'items2_proc_reordenados': items2_reordenados,
        })
        logging.info(f"Similitud '{aspecto}' calculada: Cobertura E1→E2: {metricas.get('cobertura_prom_max_1',0.0):.2%}")

    except Exception as e:
        logging.error(f"Error calculando similitud para '{aspecto}': {e}", exc_info=True)
        resultados['error'] = str(e)
    return resultados

# NUEVA FUNCIÓN específica para experiencia con jerarquía
def calcular_similitud_experiencia_con_jerarquia(
    texto1: str,
    texto2: str,
    params: dict[str, object],
    nlp_model: spacy.language.Language | None,
    sbert_model: SentenceTransformer,
    device: str
) -> dict[str, object]:
    """Calcula similitud para Experiencia, aplicando primero la lógica de jerarquía de tipos."""
    resultados: dict[str, object] = {
        'items1_proc': [], 'items2_proc': [], 'matriz': np.array([]), 'metricas': {},
        'items2_proc_reordenados': [], 'heatmap_buffer': None, 'conex_buffer': None,
        'error': None, 'jerarquia_msg': None
    }

    # 1. Extraer tipos y texto limpio
    tipo_base, texto_limpio_base = extraer_tipo_y_texto_experiencia(texto1)
    tipo_destino, texto_limpio_destino = extraer_tipo_y_texto_experiencia(texto2)

    # 2. Verificar jerarquía
    jerarquia_valida, msg = verificar_jerarquia_experiencia(tipo_base, tipo_destino)
    resultados['jerarquia_msg'] = msg

    if not jerarquia_valida:
        resultados['error'] = "Jerarquía de experiencia no es compatible."
        # Devolvemos un diccionario con métricas vacías pero con el mensaje de jerarquía
        return resultados

    # 3. Si la jerarquía es válida, proceder con la comparación semántica del texto restante
    # Usamos la función genérica pero con el texto ya pre-filtrado
    resultados_similitud = calcular_similitud_aspecto(
        texto_limpio_base,
        texto_limpio_destino,
        "Experiencia", # El aspecto sigue siendo Experiencia para el procesador de texto
        params,
        nlp_model,
        sbert_model,
        device
    )
    # Combinar los resultados
    resultados.update(resultados_similitud)
    # Asegurarse de que el mensaje de jerarquía no sea sobreescrito si hubo un error posterior
    if not resultados.get('jerarquia_msg'):
        resultados['jerarquia_msg'] = msg

    return resultados

# --- Variables Globales para la UI y Resultados ---
model_cache: dict[str, SentenceTransformer] = {}
results_data: dict[str, object] = {}
df_empleos: pd.DataFrame | None = None

# --- UI: Contenedores de Salida ---
status_area = widgets.Output(layout={'border': '1px solid #616161', 'background_color': '#212121', 'color': '#eeeeee', 'padding': '5px', 'max_height': '150px', 'overflow_y': 'auto'})
output_area = widgets.Output()


# --- Widget: Carga de Excel ---
upload_btn = widgets.FileUpload(
    accept='.xlsx',
    multiple=False,
    description="📂 Cargar Excel",
    style={'button_color': '#424242'}
)
lbl_archivo = widgets.HTML(value="")

def on_upload_change(change):
    """Callback cuando se carga un archivo: leer DataFrame y poblar selección."""
    global df_empleos
    with status_area:
        clear_output(wait=True)

    if not upload_btn.value:
        lbl_archivo.value = "<p style='color:#ffb74d;'>No se seleccionó archivo.</p>"
        return

    uploaded_file_info = next(iter(upload_btn.value.values()))
    file_name = uploaded_file_info['metadata']['name']
    content = uploaded_file_info['content']

    try:
        df = pd.read_excel(io.BytesIO(content))
        with status_area:
            print(f"Leyendo archivo '{file_name}'...")
    except Exception as e:
        lbl_archivo.value = f"<p style='color:#e57373;'>❌ Error al leer el Excel '{file_name}': {e}</p>"
        with status_area:
            print(f"❌ Error al leer el Excel '{file_name}': {e}")
        return

    columnas_necesarias = [
        ID_COLUMN_NAME, 'Código', 'Denominación', 'Grado', 'Nivel Jerárquico', 'Orden',
        'Naturaleza Jurídica', 'Salario', 'Requisitos Estudio',
        'Requisitos Experiencia', 'Funciones', 'Competencias Laborales',
        'Competencias Comportamentales'
    ]
    faltantes = [c for c in columnas_necesarias if c not in df.columns]

    if faltantes:
        faltantes_str = ", ".join(faltantes)
        msg = f"❌ Excel '{file_name}' no contiene columnas requeridas: {faltantes_str}."
        if ID_COLUMN_NAME in faltantes:
            msg += f" La columna '{ID_COLUMN_NAME}' es esencial para la identificación."
        lbl_archivo.value = f"<p style='color:#e57373;'>{msg}</p>"
        with status_area:
            print(msg)
        return

    df_empleos = df.copy()
    df_empleos[ID_COLUMN_NAME] = df_empleos[ID_COLUMN_NAME].astype(str)

    # Asegurar que las columnas de texto sean string y rellenar NaNs
    for col in [c for c in columnas_necesarias if c in df_empleos.columns]:
         if df_empleos[col].dtype == 'object' or pd.api.types.is_string_dtype(df_empleos[col]):
                df_empleos[col] = df_empleos[col].astype(str).fillna('')

    lbl_archivo.value = f"<p style='color:#66bb6a;'>✅ Archivo '{file_name}' cargado: {len(df_empleos)} empleos.</p>"

    opciones_tuples = []
    for _, row in df_empleos.iterrows():
        label = f"{str(row[ID_COLUMN_NAME])} - {str(row['Código'])}: {str(row['Denominación'])}"
        value = str(row[ID_COLUMN_NAME])
        opciones_tuples.append((label, value))

    base_dropdown.options = opciones_tuples
    destinos_select.options = opciones_tuples

    if opciones_tuples:
        if base_dropdown.value is None or base_dropdown.value not in [val for _, val in opciones_tuples]:
            base_dropdown.value = opciones_tuples[0][1]
        on_base_dropdown_change({'new': base_dropdown.value, 'name': 'value', 'type': 'change'})

    base_dropdown.disabled = False
    destinos_select.disabled = False
    marcar_todos_destinos_checkbox.disabled = False

    # Habilitar los siguientes pasos del acordeón
    accordion_config.layout.display = 'block'
    botones_box.layout.display = 'flex'

    with status_area:
        print(f"✅ Archivo '{file_name}' cargado y procesado: {len(df_empleos)} filas.")
        print(f"➡️ Seleccione empleo base (usando '{ID_COLUMN_NAME}'), destinos y configure el análisis.")

upload_btn.observe(on_upload_change, names='value')

# --- Widgets: Selección de Empleo Base y Destinos ---
base_dropdown = widgets.Dropdown(
    description="Empleo Base:",
    options=[],
    disabled=True,
    style={'description_width': '120px'},
    layout={'width': 'calc(100% - 10px)'}
)

marcar_todos_destinos_checkbox = widgets.Checkbox(
    description="Marcar/Desmarcar Todos los Destinos (excluyendo el Base)",
    value=True,
    indent=False,
    disabled=True,
    style={'description_width':'initial', 'margin_bottom':'5px'}
)

destinos_select = widgets.SelectMultiple(
    description="Empleos Destino:",
    options=[],
    disabled=True,
    style={'description_width': '120px'},
    layout={'width': 'calc(100% - 10px)', 'height': '120px'}
)

def on_marcar_todos_change(change):
    if destinos_select.options and df_empleos is not None:
        selected_base_opec = base_dropdown.value
        all_option_values = [val for label, val in destinos_select.options if val != selected_base_opec]
        if change['new']:
            destinos_select.value = all_option_values
        else:
            destinos_select.value = []
marcar_todos_destinos_checkbox.observe(on_marcar_todos_change, names='value')

def on_base_dropdown_change(change):
    if df_empleos is not None and change.get('new') is not None:
        new_base_id = change['new']
        all_options_tuples = []
        for _, row in df_empleos.iterrows():
            label = f"{str(row[ID_COLUMN_NAME])} - {str(row['Código'])}: {str(row['Denominación'])}"
            value = str(row[ID_COLUMN_NAME])
            all_options_tuples.append((label, value))
        new_dest_options = [(label, val) for label, val in all_options_tuples if val != new_base_id]
        current_dest_values = list(destinos_select.value)
        destinos_select.options = new_dest_options
        if marcar_todos_destinos_checkbox.value:
            destinos_select.value = [opt[1] for opt in new_dest_options]
        else:
            final_dest_values = [val for val in current_dest_values if val != new_base_id]
            destinos_select.value = final_dest_values

base_dropdown.observe(on_base_dropdown_change, names='value')


# --- Widgets: Selección de Modelos Semánticos ---
model_checkboxes = []
model_selection_widgets: dict[str, widgets.Checkbox] = {}
for mid in DEFAULT_MODEL_ORDER:
    if mid in MODEL_DESCRIPTIONS:
        cb = widgets.Checkbox(
            description=MODEL_DESCRIPTIONS[mid],
            value=(mid == DEFAULT_MODEL_ORDER[0]),
            indent=False,
            style={'description_width':'initial'}
        )
        model_checkboxes.append(cb)
        model_selection_widgets[mid] = cb

modelo_box = widgets.VBox(
    [widgets.HTML("<b>Modelos Semánticos (Seleccione al menos uno):</b>")] + model_checkboxes
)

# --- Widgets: Umbrales y Configuración de Similitud ---
umbral_func_slider = widgets.FloatSlider(
    description='Umbral Funciones ≥', value=DEFAULT_UMBRAL_DECISION_FUNCIONES, min=0, max=1, step=0.01, readout_format='.2f',
    style={'description_width':'180px'}, layout={'width':'98%'}
)
umbral_estudio_slider = widgets.FloatSlider(
    description='Umbral Requisitos Estudio ≥', value=DEFAULT_UMBRAL_DECISION_REQUISITOS_ESTUDIO, min=0, max=1, step=0.01, readout_format='.2f',
    style={'description_width':'180px'}, layout={'width':'98%'}
)
umbral_req_comp_slider = widgets.FloatSlider(
    description='Umbral Comp./Exp. Info. ≥', value=DEFAULT_UMBRAL_DECISION_REQUISITOS_COMP, min=0, max=1, step=0.01, readout_format='.2f',
    style={'description_width':'180px'}, layout={'width':'98%'}
)
umbral_cob_bin_slider = widgets.FloatSlider(
    description='Umbral Cob. Binaria ≥', value=DEFAULT_UMBRAL_COBERTURA_BIN, min=0, max=1, step=0.01, readout_format='.2f',
    style={'description_width':'180px'}, layout={'width':'98%'}
)
umbral_conex_slider = widgets.FloatSlider(
    description='Umbral Diag. Conexiones ≥', value=DEFAULT_UMBRAL_CONEXION, min=0, max=1, step=0.01, readout_format='.2f',
    style={'description_width':'180px'}, layout={'width':'98%'}
)
topN_slider = widgets.IntSlider(
    description='Top N Coincidencias:', value=DEFAULT_TOP_N, min=1, max=5, step=1,
    style={'description_width':'180px'}, layout={'width':'98%'}
)

umbrales_box = widgets.VBox([
    widgets.HTML("<b>Umbrales de Decisión y Visualización:</b>"),
    umbral_func_slider, umbral_estudio_slider, umbral_req_comp_slider, umbral_cob_bin_slider, umbral_conex_slider, topN_slider
])


# --- Widgets: Opciones de Decisión y Pesos ---
chk_incluir_requisitos = widgets.Checkbox(
    value=True,
    description='Incluir Requisitos (Estudio, Competencias) en la Decisión Final Automatizada',
    indent=False, style={'description_width': 'initial'}
)

peso_func_slider = widgets.FloatSlider(
    description='Funciones:', value=DEFAULT_WEIGHT_FUNCIONES, min=0.0, max=1.0, step=0.05, readout_format='.2f',
    layout={'width':'24%'}, style={'description_width':'auto'}
)
peso_edu_slider = widgets.FloatSlider(
    description='Estudio:', value=DEFAULT_WEIGHT_ESTUDIO, min=0.0, max=1.0, step=0.05, readout_format='.2f',
    layout={'width':'24%'}, style={'description_width':'auto'}
)
peso_comp_lab_slider = widgets.FloatSlider(
    description='C. Laborales:', value=DEFAULT_WEIGHT_COMP_LAB, min=0.0, max=1.0, step=0.05, readout_format='.2f',
    layout={'width':'24%'}, style={'description_width':'auto'}
)
peso_comp_comp_slider = widgets.FloatSlider(
    description='C. Comport.:', value=DEFAULT_WEIGHT_COMP_COMP, min=0.0, max=1.0, step=0.05, readout_format='.2f',
    layout={'width':'24%'}, style={'description_width':'auto'}
)

_block_weights_update = False
def _update_decision_weights(change):
    global _block_weights_update
    if _block_weights_update or not chk_incluir_requisitos.value: return
    _block_weights_update = True

    sliders = [peso_func_slider, peso_edu_slider, peso_comp_lab_slider, peso_comp_comp_slider]
    changed_slider = change.owner

    # Normalizar los pesos de los otros sliders
    other_sliders = [s for s in sliders if s != changed_slider]
    sum_of_others_before = sum(s.value for s in other_sliders)
    new_value = change.new

    remaining_total = 1.0 - new_value

    if sum_of_others_before > 0:
        scale = remaining_total / sum_of_others_before
        for s in other_sliders:
            s.value = round(s.value * scale, 2)
    else: # Si los otros eran cero, distribuir equitativamente
        if other_sliders:
            for s in other_sliders:
                s.value = round(remaining_total / len(other_sliders), 2)

    # Ajuste final para asegurar que la suma sea exactamente 1.0
    final_sum = sum(s.value for s in sliders)
    diff = 1.0 - final_sum
    # Aplicar la diferencia al slider que no fue el que cambió, si es posible
    # o al que cambió si es el único con valor > 0
    if diff != 0:
        non_changed_sliders = [s for s in sliders if s != changed_slider and s.value > 0]
        if non_changed_sliders:
            non_changed_sliders[0].value = round(non_changed_sliders[0].value + diff, 2)
        else:
            changed_slider.value = round(changed_slider.value + diff, 2)

    _block_weights_update = False

for s in [peso_func_slider, peso_edu_slider, peso_comp_lab_slider, peso_comp_comp_slider]:
    s.observe(_update_decision_weights, 'value')

def on_incluir_reqs_change(change):
    global _block_weights_update
    enabled = change['new']
    _block_weights_update = True

    peso_edu_slider.disabled = not enabled
    peso_comp_lab_slider.disabled = not enabled
    peso_comp_comp_slider.disabled = not enabled

    if not enabled:
        peso_func_slider.value = 1.0
        peso_edu_slider.value = 0.0
        peso_comp_lab_slider.value = 0.0
        peso_comp_comp_slider.value = 0.0
    else:
        # Restaurar pesos por defecto
        peso_func_slider.value = DEFAULT_WEIGHT_FUNCIONES
        peso_edu_slider.value = DEFAULT_WEIGHT_ESTUDIO
        peso_comp_lab_slider.value = DEFAULT_WEIGHT_COMP_LAB
        peso_comp_comp_slider.value = DEFAULT_WEIGHT_COMP_COMP

    _block_weights_update = False
chk_incluir_requisitos.observe(on_incluir_reqs_change, names='value')

decision_weights_box = widgets.HBox([peso_func_slider, peso_edu_slider, peso_comp_lab_slider, peso_comp_comp_slider], layout={'justify_content':'space-between'})
decision_box = widgets.VBox([widgets.HTML("<b>Ponderación para Decisión Final (Suma 1.0):</b>"), chk_incluir_requisitos, decision_weights_box])


# --- Widgets: Opciones de Texto y Jaccard ---
elim_dups_box = widgets.Checkbox(description='Eliminar Duplicados', value=True, indent=False, style={'description_width':'initial'})
norm_box = widgets.Checkbox(description='Normalizar Texto', value=True, indent=False, style={'description_width':'initial'})
lemma_box = widgets.Checkbox(
    description=f'Lematizar (spaCy {"OK" if SPACY_AVAILABLE else "NO DISPONIBLE"})',
    value=False, indent=False, style={'description_width':'initial'}, disabled=not SPACY_AVAILABLE
)
gpu_box = widgets.Checkbox(
    description=f'Usar GPU (CUDA {"Disponible" if torch.cuda.is_available() else "No Disponible"})',
    value=torch.cuda.is_available(), indent=False, disabled=not torch.cuda.is_available(), style={'description_width':'initial'}
)
usar_jacc_box = widgets.Checkbox(description='Combinar Coseno+Jaccard', value=False, indent=False, style={'description_width':'initial'})

peso_cos_slider = widgets.FloatSlider(
    description='Peso Coseno:', value=DEFAULT_PESO_COSENO, min=0.0, max=1.0, step=0.05, readout_format='.2f',
    layout={'width':'48%'}, style={'description_width':'auto'}, disabled=True
)
peso_jacc_slider = widgets.FloatSlider(
    description='Peso Jaccard:', value=DEFAULT_PESO_JACCARD, min=0.0, max=1.0, step=0.05, readout_format='.2f',
    layout={'width':'48%'}, style={'description_width':'auto'}, disabled=True
)

_block_jaccard_update = False
def on_jaccard_slider_change(main_slider, other_slider, change):
    global _block_jaccard_update
    if _block_jaccard_update: return
    _block_jaccard_update = True
    other_slider.value = round(1.0 - change['new'], 2)
    _block_jaccard_update = False

def on_usar_jacc_change(change):
    is_enabled = change['new']
    peso_cos_slider.disabled = not is_enabled
    peso_jacc_slider.disabled = not is_enabled

peso_cos_slider.observe(lambda c: on_jaccard_slider_change(peso_cos_slider, peso_jacc_slider, c), names='value')
peso_jacc_slider.observe(lambda c: on_jaccard_slider_change(peso_jacc_slider, peso_cos_slider, c), names='value')
usar_jacc_box.observe(on_usar_jacc_change, names='value')

jacc_sliders_box = widgets.HBox([peso_cos_slider, peso_jacc_slider], layout={'justify_content':'space-between'})
jacc_options_box = widgets.VBox([usar_jacc_box, jacc_sliders_box])

procesamiento_texto_box = widgets.VBox([
    widgets.HTML("<b>Procesamiento de Texto y Similitud:</b>"),
    widgets.HBox([elim_dups_box, norm_box, lemma_box, gpu_box], layout={'flex_wrap':'wrap'}),
    widgets.HTML("<hr style='border-color:#4a4a4a; margin:4px 0;'/>"),
    jacc_options_box
])

# --- Widgets: Información Adicional para Informe ---
proceso_input = widgets.Text(description='Proceso (Opcional PDF):', placeholder='Ej: Convocatoria XYZ', style={'description_width': '180px'}, layout={'width':'98%'})
analista_input = widgets.Text(description='Analista (Opcional PDF):', placeholder='Tu nombre', style={'description_width': '180px'}, layout={'width':'98%'})
profesion_aspirante_input = widgets.Text(description='Profesión Aspirante/Servidor:', placeholder='Ej: Ingeniero de Sistemas', style={'description_width': '180px'}, layout={'width':'98%'})

tipo_uso_dropdown = widgets.Dropdown(
    description='Tipo de Uso del Estudio:',
    options=LISTA_TIPOS_USO,
    value=LISTA_TIPOS_USO[0],
    style={'description_width': '180px'}, layout={'width':'98%'}
)
tipo_uso_otro_input = widgets.Text(
    description='Cuál (si Otro):',
    placeholder='Especifique el tipo de uso',
    style={'description_width': '180px'}, layout={'width':'98%', 'display':'none'}
)

def on_tipo_uso_change(change):
    if change['new'] == TipoUsoEstudio.OTRO.value:
        tipo_uso_otro_input.layout.display = 'block'
    else:
        tipo_uso_otro_input.layout.display = 'none'
tipo_uso_dropdown.observe(on_tipo_uso_change, names='value')

info_adicional_box = widgets.VBox(
    [widgets.HTML("<b>Información Adicional (para PDF):</b>"),
     proceso_input, analista_input, profesion_aspirante_input,
     tipo_uso_dropdown, tipo_uso_otro_input
    ]
)

# --- Contenedor de Configuración Mejorado ---
config_vbox_style = {'border': f'1px solid {AppColors.ACCENT_BORDER.value.hexval()}', 'padding': '8px', 'margin_top': '10px', 'border_radius': '5px'}
accordion_config = widgets.VBox(
    [
        widgets.VBox([modelo_box], layout=config_vbox_style),
        widgets.VBox([umbrales_box], layout=config_vbox_style),
        widgets.VBox([decision_box], layout=config_vbox_style),
        widgets.VBox([procesamiento_texto_box], layout=config_vbox_style),
        widgets.VBox([info_adicional_box], layout=config_vbox_style),
    ],
    layout={'display': 'none'} # Inicialmente oculto hasta cargar archivo
)

# --- Widgets: Botones de Ejecución y Exportación ---
boton_analizar = widgets.Button(
    description="🔍 Analizar Equivalencia", button_style='info', icon='cogs', tooltip='Iniciar análisis completo',
    layout={'width': 'auto', 'flex': '1 1 auto'}, style={'button_color': '#4fc3f7'}
)
boton_exportar_pdf = widgets.Button(
    description="📄 Exportar PDF", button_style='success', icon='file-pdf', tooltip='Generar y descargar PDF',
    layout={'width': 'auto', 'flex': '1 1 auto'}, style={'button_color': '#66bb6a'}, disabled=True
)
boton_exportar_excel = widgets.Button(
    description="📊 Exportar Excel", button_style='warning', icon='file-excel-o', tooltip='Generar y descargar Excel',
    layout={'width': 'auto', 'flex': '1 1 auto'}, style={'button_color': '#ffb74d'}, disabled=True
)
botones_box = widgets.HBox(
    [boton_analizar, boton_exportar_pdf, boton_exportar_excel],
    layout={'display': 'none', 'justify_content': 'space-around', 'margin_top': '10px', 'gap': '10px'}
)


# --- Función Principal: on_analizar_click ---
def on_analizar_click(b):
    """Ejecuta todo el flujo de análisis de equivalencia para múltiples destinos."""
    global results_data, model_cache

    output_accordion.children = []
    output_accordion.selected_index = None
    with status_area:
        clear_output(wait=True)
        print("🔄 Iniciando análisis de equivalencia...")

    boton_exportar_pdf.disabled = True
    boton_exportar_excel.disabled = True
    results_data = {}

    # --- Validaciones de Entrada ---
    if df_empleos is None:
        with status_area:
            print("⚠️ Error: No hay ningún Excel cargado.")
        return
    if not base_dropdown.value:
        with status_area:
            print("⚠️ Error: Selecciona un Empleo Base.")
        return
    if not destinos_select.value:
        with status_area:
            print("⚠️ Error: Selecciona al menos un Empleo Destino.")
        return

    selected_model_ids_from_ui = [mid for mid, cb_widget in model_selection_widgets.items() if cb_widget.value]
    if not selected_model_ids_from_ui:
        with status_area:
            print("⚠️ Error: Selecciona al menos un modelo semántico.")
        return

    try:
        opec_base_id = str(base_dropdown.value)
        df_base_row_series = df_empleos[df_empleos[ID_COLUMN_NAME].astype(str) == opec_base_id]
        if df_base_row_series.empty:
            with status_area:
                print(f"⚠️ Error: Empleo Base con {ID_COLUMN_NAME} '{opec_base_id}' no encontrado.")
            return
        df_base_row = df_base_row_series.iloc[0]
    except Exception as e:
        with status_area:
            print(f"⚠️ Error al obtener Empleo Base '{opec_base_id}': {e}")
        return

    opec_destinos_ids = [str(val) for val in destinos_select.value if str(val) != opec_base_id]
    if not opec_destinos_ids:
        with status_area:
            print("⚠️ Error: No hay destinos válidos seleccionados (diferentes al base).")
        return

    df_destinos_rows = df_empleos[df_empleos[ID_COLUMN_NAME].astype(str).isin(opec_destinos_ids)]
    if df_destinos_rows.empty:
        with status_area:
            print("⚠️ Error: Ningún empleo destino válido encontrado en el Excel.")
        return

    params_dict: dict[str, object] = {
        ID_COLUMN_NAME: df_base_row.get(ID_COLUMN_NAME), 'Código': df_base_row.get('Código'),
        'Denominación': df_base_row.get('Denominación'), 'Grado': int(df_base_row.get('Grado', 0)),
        'Nivel Jerárquico': str(df_base_row.get('Nivel Jerárquico')), 'Orden': str(df_base_row.get('Orden')),
        'Naturaleza Jurídica': str(df_base_row.get('Naturaleza Jurídica')), 'Salario': float(df_base_row.get('Salario', 0.0)),
        'Requisitos Estudio Base': str(df_base_row.get('Requisitos Estudio')),
        'Requisitos Experiencia Base': str(df_base_row.get('Requisitos Experiencia')),
        'Funciones Base': str(df_base_row.get('Funciones')),
        'Competencias Laborales Base': str(df_base_row.get('Competencias Laborales')),
        'Competencias Comportamentales Base': str(df_base_row.get('Competencias Comportamentales')),
        'selected_model_ids_from_ui': selected_model_ids_from_ui, 'loaded_model_ids': [],
        'umbral_decision_funciones': umbral_func_slider.value,
        'umbral_decision_requisitos_comp': umbral_req_comp_slider.value,
        'umbral_decision_requisitos_estudio': umbral_estudio_slider.value,
        'coverage_threshold_bin': umbral_cob_bin_slider.value, 'umbral_conexion': umbral_conex_slider.value,
        'top_n': topN_slider.value, 'eliminar_dups': elim_dups_box.value, 'normalizar': norm_box.value,
        'lematizar': lemma_box.value and SPACY_AVAILABLE, 'use_gpu': gpu_box.value and torch.cuda.is_available(),
        'min_words': MIN_WORDS_PER_ITEM, 'usar_jaccard': usar_jacc_box.value,
        'peso_cos': peso_cos_slider.value if usar_jacc_box.value else 1.0,
        'peso_jacc': peso_jacc_slider.value if usar_jacc_box.value else 0.0,
        'Proceso Selección': proceso_input.value.strip(), 'Analista': analista_input.value.strip(),
        'Profesion Aspirante': profesion_aspirante_input.value.strip(), 'Tipo de Uso': tipo_uso_dropdown.value,
        'Tipo de Uso Otro': tipo_uso_otro_input.value.strip() if tipo_uso_dropdown.value == TipoUsoEstudio.OTRO.value else "",
        'Fecha': datetime.now().strftime("%d/%m/%Y"),
        # NUEVO: Guardar estado del checkbox de requisitos
        'incluir_requisitos_en_decision': chk_incluir_requisitos.value,
        'peso_funciones_decision': peso_func_slider.value, 'peso_estudio_decision': peso_edu_slider.value,
        'peso_comp_lab_decision': peso_comp_lab_slider.value, 'peso_comp_comp_decision': peso_comp_comp_slider.value,
    }
    results_data['params'] = params_dict
    results_data['results_por_destino'] = {}
    results_data['raw_base'] = df_base_row

    device_str = 'cuda' if params_dict['use_gpu'] else 'cpu'
    with status_area: print(f"🖥️ Usando dispositivo: {device_str.upper()}")

    loaded_models_ids = []
    for mid in selected_model_ids_from_ui:
        if mid not in model_cache or model_cache.get(mid) is None:
            try:
                with status_area: print(f"⏳ Cargando modelo '{MODEL_DESCRIPTIONS.get(mid, mid)}' en {device_str.upper()}...")
                model_cache[mid] = SentenceTransformer(mid, device=device_str)
                loaded_models_ids.append(mid)
                with status_area: print(f"✅ Modelo '{MODEL_DESCRIPTIONS.get(mid, mid)}' cargado.")
            except Exception as e:
                with status_area: print(f"❌ Error crítico cargando modelo '{mid}': {e}. Este modelo no será usado.");
                model_cache[mid] = None
        elif model_cache[mid] is not None:
            loaded_models_ids.append(mid)

    params_dict['loaded_model_ids'] = loaded_models_ids
    if not params_dict['loaded_model_ids']:
        with status_area:
            print("❌ Ningún modelo semántico pudo ser cargado. Análisis detenido.")
        return

    nlp_to_use = nlp if params_dict['lematizar'] else None

    output_children = []

    for _, fila_dest in df_destinos_rows.iterrows():
        opec_dest = str(fila_dest.get(ID_COLUMN_NAME))
        key_dest = f"{opec_dest} - {fila_dest.get('Código')}: {fila_dest.get('Denominación')}"
        with status_area: print(f"\n--- 🎯 Analizando Destino: {key_dest} ---")

        target_results: dict[str, object] = { 'target_OPEC': opec_dest, 'target_Codigo': fila_dest.get('Código'), 'target_Denominacion': fila_dest.get('Denominación'), 'raw_dest': fila_dest }
        target_results['nivel'] = comparar_niveles(str(params_dict['Nivel Jerárquico']), str(fila_dest.get('Nivel Jerárquico')))
        target_results['salario'] = comparar_salarios(float(params_dict.get('Salario', 0.0)), float(fila_dest.get('Salario', 0.0)), int(params_dict.get('Grado', -1)), int(fila_dest.get('Grado', -1)), str(params_dict.get('Orden')), str(params_dict.get('Naturaleza Jurídica')), str(fila_dest.get('Orden')), str(fila_dest.get('Naturaleza Jurídica')))

        target_results['results_per_model'] = {}
        ref_model_id: str | None = None

        for model_id in DEFAULT_MODEL_ORDER:
            if model_id not in params_dict['loaded_model_ids']: continue
            sbert_model = model_cache.get(model_id)
            if not sbert_model: continue

            model_results: dict[str, object] = {'model_desc': MODEL_DESCRIPTIONS.get(model_id, model_id), 'error_general': None}
            with status_area: print(f"--- Modelo: {model_results['model_desc']} ---")
            try:
                model_results['similitud_funciones'] = calcular_similitud_aspecto(str(params_dict['Funciones Base']), str(fila_dest['Funciones']), 'Funciones', params_dict, nlp_to_use, sbert_model, device_str)
                model_results['similitud_educacion'] = calcular_similitud_aspecto(str(params_dict['Requisitos Estudio Base']), str(fila_dest['Requisitos Estudio']), 'Educacion', params_dict, nlp_to_use, sbert_model, device_str)
                model_results['similitud_comp_lab'] = calcular_similitud_aspecto(str(params_dict['Competencias Laborales Base']), str(fila_dest['Competencias Laborales']), 'Competencias Laborales', params_dict, nlp_to_use, sbert_model, device_str)
                model_results['similitud_comp_comp'] = calcular_similitud_aspecto(str(params_dict['Competencias Comportamentales Base']), str(fila_dest['Competencias Comportamentales']), 'Competencias Comportamentales', params_dict, nlp_to_use, sbert_model, device_str)
                # NUEVO: Llamada a función de experiencia con jerarquía
                model_results['similitud_experiencia'] = calcular_similitud_experiencia_con_jerarquia(str(params_dict['Requisitos Experiencia Base']), str(fila_dest['Requisitos Experiencia']), params_dict, nlp_to_use, sbert_model, device_str)

                if not model_results['similitud_funciones'].get('error') and model_results['similitud_funciones'].get('metricas') and ref_model_id is None:
                    ref_model_id = model_id
                    with status_area: print(f"➡️ {model_results['model_desc']} establecido como modelo de referencia para {key_dest}.")
                    # Preparar datos para gráficos
                    plot_aspects_data = {
                        'similitud_funciones': (model_results.get('similitud_funciones',{}), "Funciones", "F1", "F2", None),
                        'similitud_educacion': (model_results.get('similitud_educacion',{}), "Educación", "Ed1", "Ed2", None),
                        'similitud_experiencia': (model_results.get('similitud_experiencia',{}), "Experiencia", "Ex1", "Ex2", model_results.get('similitud_experiencia', {}).get('jerarquia_msg')),
                        'similitud_comp_lab': (model_results.get('similitud_comp_lab',{}), "Comp. Laborales", "CL1", "CL2", None),
                        'similitud_comp_comp': (model_results.get('similitud_comp_comp',{}), "Comp. Comportamentales", "CC1", "CC2", None)
                    }
                    target_results['plot_data'] = plot_aspects_data # Guardar para el HTML
                    # Generar gráficos
                    for key, (sim_data, titulo, p1, p2, _) in plot_aspects_data.items():
                        if sim_data and sim_data.get('items1_proc') and sim_data.get('items2_proc_reordenados'):
                            sim_data['heatmap_buffer'] = generar_heatmap(sim_data.get('matriz', np.array([])), sim_data.get('items1_proc', []), sim_data.get('items2_proc_reordenados', []), f"Matriz Similitud {titulo}", prefix_1=p1, prefix_2=p2)
                            sim_data['conex_buffer'] = generar_diagrama_conexiones(sim_data.get('items1_proc', []), sim_data.get('items2_proc_reordenados', []), sim_data.get('matriz', np.array([])), f"Conexiones {titulo}", prefix_1=p1, prefix_2=p2, umbral_conexion=float(params_dict['umbral_conexion']))

            except Exception as ex_model:
                model_results['error_general'] = str(ex_model)
                with status_area: print(f"❌ Error procesando con modelo '{model_results['model_desc']}': {ex_model}")
            finally:
                target_results['results_per_model'][model_id] = model_results
                if device_str == 'cuda': torch.cuda.empty_cache()

        target_results['reference_model_id'] = ref_model_id
        razones_no_eq: list[str] = []
        if not target_results.get('nivel', {}).get('pasa'): razones_no_eq.append(str(target_results.get('nivel', {}).get('mensaje','Falla Nivel')))
        if not target_results.get('salario', {}).get('pasa'): razones_no_eq.append(str(target_results.get('salario', {}).get('mensaje','Falla Salario')))

        # Evaluar criterios de decisión
        def eval_crit(sim_data, umbral, label, raw1, raw2):
            if not _is_valid_data(params_dict[raw1]) and not _is_valid_data(fila_dest[raw2]): return True, None, 1.0 # Pasa si no hay datos en ninguno
            if not _is_valid_data(params_dict[raw1]) or not _is_valid_data(fila_dest[raw2]): return False, f"{label}: Datos válidos incompletos.", 0.0
            if sim_data.get('error'): return False, f"Error en {label}: {str(sim_data['error'])[:60]}...", 0.0
            if not sim_data.get('metricas'): return False, f"{label}: Sin métricas de similitud.", 0.0
            cob = sim_data.get('metricas', {}).get('cobertura_prom_max_1', 0.0)
            pasa = cob >= umbral
            razon = f"Similitud {cob:.1%} < umbral ({umbral:.1%}) para {label}." if not pasa else None
            return pasa, razon, cob

        pasa_f, r_f, s_f = (False, "Sin modelo de referencia", 0.0)
        pasa_e, r_e, s_e = (False, "Sin modelo de referencia", 0.0)
        pasa_l, r_l, s_l = (False, "Sin modelo de referencia", 0.0)
        pasa_c, r_c, s_c = (False, "Sin modelo de referencia", 0.0)

        if ref_model_id and ref_model_id in target_results['results_per_model']:
            ref_res = target_results['results_per_model'][ref_model_id]
            ref_desc = ref_res.get('model_desc','REF').split(' (')[0]

            pasa_f, r_f, s_f = eval_crit(ref_res.get('similitud_funciones',{}), params_dict['umbral_decision_funciones'], 'Funciones', 'Funciones Base', 'Funciones')
            pasa_e, r_e, s_e = eval_crit(ref_res.get('similitud_educacion',{}), params_dict['umbral_decision_requisitos_estudio'], 'Requisitos Estudio', 'Requisitos Estudio Base', 'Requisitos Estudio')
            pasa_l, r_l, s_l = eval_crit(ref_res.get('similitud_comp_lab',{}), params_dict['umbral_decision_requisitos_comp'], 'Comp. Laborales', 'Competencias Laborales Base', 'Competencias Laborales')
            pasa_c, r_c, s_c = eval_crit(ref_res.get('similitud_comp_comp',{}), params_dict['umbral_decision_requisitos_comp'], 'Comp. Comportamentales', 'Competencias Comportamentales Base', 'Competencias Comportamentales')

            if r_f: razones_no_eq.append(f"{r_f} (Mod: {ref_desc})")
            if r_e: razones_no_eq.append(f"{r_e} (Mod: {ref_desc})")
            if r_l: razones_no_eq.append(f"{r_l} (Mod: {ref_desc})")
            if r_c: razones_no_eq.append(f"{r_c} (Mod: {ref_desc})")

        target_results['pasa_funciones'] = pasa_f
        target_results['pasa_educacion_decision'] = pasa_e
        target_results['pasa_comp_lab'] = pasa_l
        target_results['pasa_comp_comp'] = pasa_c

        # Decisión final
        es_equiv = target_results['nivel']['pasa'] and target_results['salario']['pasa'] and pasa_f
        if params_dict['incluir_requisitos_en_decision']:
            es_equiv = es_equiv and pasa_e and pasa_l and pasa_c

        # Puntuación final
        if params_dict['incluir_requisitos_en_decision']:
            w_sum = (s_f * params_dict['peso_funciones_decision'] + s_e * params_dict['peso_estudio_decision'] +
                     s_l * params_dict['peso_comp_lab_decision'] + s_c * params_dict['peso_comp_comp_decision'])
            w_total = (params_dict['peso_funciones_decision'] + params_dict['peso_estudio_decision'] +
                       params_dict['peso_comp_lab_decision'] + params_dict['peso_comp_comp_decision'])
            score = w_sum / w_total if w_total > 0 else 0.0
        else:
            score = s_f # Solo cuenta funciones

        target_results['es_equivalente'] = es_equiv
        target_results['equivalence_score'] = score # El score se muestra siempre
        target_results['razones_no_equivalencia'] = sorted(list(set(r for r in razones_no_eq if r)))
        results_data['results_por_destino'][key_dest] = target_results

        # Preparar HTML para el acordeón de salida
        html_content = ""
        if ref_model_id:
            ref_model_results = target_results['results_per_model'][ref_model_id]
            html_summary_data = {
                'params': params_dict, 'equivalencia_results': target_results,
                'similitud_funciones': ref_model_results.get('similitud_funciones', {}),
                'similitud_educacion': ref_model_results.get('similitud_educacion', {}),
                'similitud_experiencia': ref_model_results.get('similitud_experiencia', {}),
                'similitud_comp_lab': ref_model_results.get('similitud_comp_lab', {}),
                'similitud_comp_comp': ref_model_results.get('similitud_comp_comp', {}),
                'results_per_model': target_results['results_per_model'],
                'reference_model_id': ref_model_id
            }
            html_content += mostrar_resumen_equivalencia_html(html_summary_data)
            html_content += generar_tabla_comparacion_global_html(html_summary_data)

            if 'plot_data' in target_results:
                for key, (sim_data, titulo, p1, p2, jerarquia_msg) in target_results['plot_data'].items():
                    html_content += mostrar_detalles_similitud_html(
                        f"Detalle Similitud {titulo}",
                        sim_data.get('items1_proc', []), sim_data.get('items2_proc_reordenados', []),
                        sim_data.get('matriz', np.array([])), p1, p2,
                        int(params_dict.get('top_n', 3)), float(params_dict.get('coverage_threshold_bin', 0.6)),
                        ref_desc, info_extra_header=jerarquia_msg
                    )
        else:
            html_content = f"<p style='color:{AppColors.WARNING.value.hexval()};'><i>No se pudo determinar un modelo de referencia válido para detalles y gráficos de '{key_dest}'.</i></p>"
            # Aún así, mostrar resumen básico
            html_content += mostrar_resumen_equivalencia_html({
                'params': params_dict, 'equivalencia_results': target_results,
                'reference_model_id': None, 'results_per_model': {}
            })

        output_children.append(widgets.HTML(html_content))

    output_accordion.children = output_children
    for i, key_dest in enumerate(results_data.get('results_por_destino', {}).keys()):
        output_accordion.set_title(i, key_dest)

    with status_area:
        print("\n🏁 Análisis finalizado para todos los destinos.")
        if results_data.get('results_por_destino'):
            print("✅ Puedes exportar los informes en PDF o Excel.")
            boton_exportar_pdf.disabled = False
            boton_exportar_excel.disabled = False
            if output_accordion.children:
                output_accordion.selected_index = 0
        else:
            print("⚠️ Ningún destino pudo ser analizado. No hay informes para exportar.")

boton_analizar.on_click(on_analizar_click)

# --- Funciones de Exportación (COMPLETAS) ---

def generar_grafico_resumen_puntuaciones(destinos_res_dict: dict) -> io.BytesIO | None:
    """NUEVA FUNCIÓN: Genera un gráfico de barras horizontales con las puntuaciones de equivalencia."""
    import matplotlib.pyplot as plt

    if not destinos_res_dict:
        return None

    labels = []
    scores = []
    colors = []

    # Extraer datos y ordenar por puntuación descendente
    sorted_destinos = sorted(
        destinos_res_dict.values(),
        key=lambda x: x.get('equivalence_score', 0.0),
        reverse=True
    )

    for res in sorted_destinos:
        target_opec = res.get('target_OPEC', 'N/A')
        target_denom = res.get('target_Denominacion', 'N/A')

        labels.append(f"{target_opec} - {truncar_texto(target_denom, 35)}")
        scores.append(res.get('equivalence_score', 0.0))
        colors.append('#66bb6a' if res.get('es_equivalente') else '#e57373') # Verde/Rojo del tema

    if not labels:
        return None

    num_items = len(labels)
    # Ajustar altura de la figura dinámicamente
    fig_height = max(4, num_items * 0.5)
    fig, ax = plt.subplots(figsize=(10, fig_height), dpi=120)

    y_pos = np.arange(num_items)
    bars = ax.barh(y_pos, scores, align='center', color=colors, height=0.6)

    ax.set_yticks(y_pos)
    ax.set_yticklabels(labels, fontsize=8)
    ax.invert_yaxis()  # El de mayor puntaje arriba
    ax.set_xlabel('Puntuación de Equivalencia (Ponderada)', fontsize=9)
    ax.set_title('Resumen Visual de Puntuaciones de Equivalencia', fontsize=12, pad=15)
    ax.set_xlim(0, 1) # Las puntuaciones son de 0 a 1
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}')) # Formato de porcentaje

    # Añadir las etiquetas con el valor de la puntuación en cada barra
    for bar in bars:
        width = bar.get_width()
        ax.text(
            width + 0.01,
            bar.get_y() + bar.get_height() / 2,
            f'{width:.1%}',
            va='center',
            ha='left',
            fontsize=7
        )

    fig.tight_layout()

    buf = io.BytesIO()
    try:
        fig.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        plt.close(fig)
        return buf
    except Exception as e:
        logging.error(f"Error al generar gráfico de puntuaciones: {e}", exc_info=True)
        if 'fig' in locals() and fig: plt.close(fig)
        return None


def generar_tabla_resumen_global_pdf(
    destinos_res_dict: dict,
    styles: dict,
    pdf_text_color: reportlab_colors.Color,
    pdf_success_color: reportlab_colors.Color,
    pdf_danger_color: reportlab_colors.Color,
    pdf_neutral_color: reportlab_colors.Color,
    pdf_warning_color: reportlab_colors.Color,
    pdf_grid_color: reportlab_colors.Color,
    pdf_accent_bg_color: reportlab_colors.Color
    ) -> Table | None:
    if not destinos_res_dict: return None
    header_style_pdf = ParagraphStyle(name='HeaderResumenPDF', parent=styles['Normal'], fontName='Helvetica-Bold', fontSize=7, textColor=pdf_text_color, alignment=TA_CENTER)
    cell_style_pdf = ParagraphStyle(name='CellResumenPDF', parent=styles['Normal'], fontSize=6, textColor=pdf_text_color, alignment=TA_CENTER)
    pasa_style_pdf_table = ParagraphStyle(name='PasaTable', parent=cell_style_pdf, textColor=pdf_success_color)
    no_pasa_style_pdf_table = ParagraphStyle(name='NoPasaTable', parent=cell_style_pdf, textColor=pdf_danger_color)
    na_style_pdf_table = ParagraphStyle(name='NATable', parent=cell_style_pdf, textColor=pdf_neutral_color, fontName='Helvetica-Oblique')
    info_style_pdf_table = ParagraphStyle(name='InfoTable', parent=cell_style_pdf, textColor=pdf_warning_color)

    header_data = [
        Paragraph("Empleo Destino<br/>(OPEC - Código: Denom.)", header_style_pdf),
        Paragraph("Nivel", header_style_pdf), Paragraph("Salario/<br/>Grado", header_style_pdf),
        Paragraph("Funciones<br/>(Ref)", header_style_pdf), Paragraph("Educación<br/>(Ref)", header_style_pdf),
        Paragraph("Experiencia<br/>(Info. Ref)", header_style_pdf), Paragraph("C. Lab.<br/>(Ref)", header_style_pdf),
        Paragraph("C. Comp.<br/>(Ref)", header_style_pdf), Paragraph("Decisión<br/>Automatizada", header_style_pdf)
    ]
    table_data_pdf = [header_data]

    for key_dest, res_dest in destinos_res_dict.items():
        target_opec = res_dest.get('target_OPEC', 'N/A')
        target_codigo = res_dest.get('target_Codigo', 'N/A')
        target_denom = res_dest.get('target_Denominacion', 'N/A')
        empleo_destino_str = f"{target_opec} - {target_codigo}: {truncar_texto(target_denom, 25)}"

        # Determinar si los requisitos fueron parte de la decisión
        reqs_en_decision = results_data.get('params', {}).get('incluir_requisitos_en_decision', True)

        def get_pass_fail_na_text_pdf(pasa_value, es_informativo=False, es_requisito_opcional=False):
            if es_requisito_opcional and not reqs_en_decision:
                es_informativo = True

            if pasa_value is True:
                return Paragraph("Pasa" if not es_informativo else "Sí (Info)", pasa_style_pdf_table if not es_informativo else info_style_pdf_table)
            elif pasa_value is False:
                return Paragraph("No Pasa" if not es_informativo else "No (Info)", no_pasa_style_pdf_table if not es_informativo else info_style_pdf_table)
            return Paragraph("N/A", na_style_pdf_table)

        pasa_exp_info = None
        if ref_model_id_exp := res_dest.get('reference_model_id'):
            if sim_exp_data := res_dest.get('results_per_model', {}).get(ref_model_id_exp, {}).get('similitud_experiencia', {}):
                if not sim_exp_data.get('error'):
                        pasa_exp_info = True # Solo para colorear como informativo

        row_data = [
            Paragraph(empleo_destino_str, ParagraphStyle(name=f'DestName_{target_opec}', parent=cell_style_pdf, alignment=TA_LEFT)),
            get_pass_fail_na_text_pdf(res_dest.get('nivel', {}).get('pasa')),
            get_pass_fail_na_text_pdf(res_dest.get('salario', {}).get('pasa')),
            get_pass_fail_na_text_pdf(res_dest.get('pasa_funciones')),
            get_pass_fail_na_text_pdf(res_dest.get('pasa_educacion_decision'), es_requisito_opcional=True),
            get_pass_fail_na_text_pdf(pasa_exp_info, es_informativo=True),
            get_pass_fail_na_text_pdf(res_dest.get('pasa_comp_lab'), es_requisito_opcional=True),
            get_pass_fail_na_text_pdf(res_dest.get('pasa_comp_comp'), es_requisito_opcional=True),
            Paragraph("Equivalente (Auto)" if res_dest.get('es_equivalente') else "No Equivalente (Auto)",
                        pasa_style_pdf_table if res_dest.get('es_equivalente') else no_pasa_style_pdf_table)
        ]
        table_data_pdf.append(row_data)

    col_widths = [2.0*inch, 0.5*inch, 0.6*inch, 0.6*inch, 0.7*inch, 0.7*inch, 0.6*inch, 0.6*inch, 0.8*inch]
    available_width = letter[0] - 1.4*inch
    total_width_needed = sum(col_widths)
    if total_width_needed > available_width:
        scale = available_width / total_width_needed
        col_widths = [w * scale for w in col_widths]

    summary_table = Table(table_data_pdf, colWidths=col_widths, repeatRows=1)
    summary_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), pdf_accent_bg_color), ('GRID', (0, 0), (-1, -1), 0.5, pdf_grid_color),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'), ('LEFTPADDING', (0,0), (-1,-1), 2), ('RIGHTPADDING', (0,0), (-1,-1), 2),
        ('TOPPADDING', (0,0), (-1,-1), 2), ('BOTTOMPADDING', (0,0), (-1,-1), 2), ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('ALIGN', (1,0), (-1,-1), 'CENTER'), ('ALIGN', (0,1), (0,-1), 'LEFT'),
    ]))
    return summary_table

def add_page_bg_light(canvas, doc):
    """Dibuja rectángulo blanco de fondo en cada página del PDF."""
    canvas.saveState()
    canvas.setFillColor(reportlab_colors.white)
    canvas.rect(0, 0, letter[0], letter[1], fill=1, stroke=0)
    canvas.restoreState()

def on_exportar_click_pdf(b):
    """Genera un único PDF (Light Theme) que reúne todos los informes y lo descarga."""
    if not results_data or not results_data.get('results_por_destino'):
        with status_area: clear_output(wait=True); print("⚠️ No hay resultados disponibles para exportar a PDF."); return

    params = results_data.get('params', {})
    destinos_res_dict = results_data.get('results_por_destino', {})

    pdf_buffer = io.BytesIO()
    base_id_for_fn = str(params.get(ID_COLUMN_NAME, params.get('Código', 'Base')))
    empleo_base_filename_part = base_id_for_fn.replace(' ', '_').replace('/', '_')[:30]
    fecha_segura_filename_part = str(params.get('Fecha', datetime.now().strftime("%d-%m-%Y"))).replace('/', '-')
    pdf_filename = f"Informe_Equivalencia_{empleo_base_filename_part}_{fecha_segura_filename_part}.pdf"

    doc = SimpleDocTemplate(
        pdf_buffer, pagesize=letter,
        leftMargin=0.7*inch, rightMargin=0.7*inch,
        topMargin=0.7*inch, bottomMargin=0.7*inch,
        title=f"Informe Múltiple Equivalencia {params.get('Denominación','Base')}",
        author="Analizador de Equivalencia de Empleos"
    )

    pdf_text_color = reportlab_colors.black
    pdf_text_light_color = reportlab_colors.HexColor('#333333')
    pdf_grid_color = reportlab_colors.lightgrey
    pdf_accent_bg_color = reportlab_colors.HexColor('#F0F0F0')
    pdf_primary_h_color = reportlab_colors.HexColor('#0D47A1')

    pdf_secondary_h_color = reportlab_colors.HexColor('#4A148C')
    pdf_success_color = reportlab_colors.HexColor('#1B5E20')
    pdf_danger_color = reportlab_colors.HexColor('#B71C1C')
    pdf_warning_color = reportlab_colors.HexColor('#E65100')
    pdf_neutral_color = reportlab_colors.HexColor('#424242')

    styles = getSampleStyleSheet()
    styles['Normal'].fontName = 'Helvetica'; styles['Normal'].fontSize = 9; styles['Normal'].leading = 11; styles['Normal'].textColor = pdf_text_color
    styles.add(ParagraphStyle(name='BodyJustify', parent=styles['Normal'], alignment=TA_JUSTIFY))
    styles.add(ParagraphStyle(name='SmallBodyJustify', parent=styles['Normal'], alignment=TA_JUSTIFY, fontSize=8, leading=10))
    styles['Heading1'].fontName = 'Helvetica-Bold'; styles['Heading1'].fontSize = 14; styles['Heading1'].textColor = pdf_primary_h_color; styles['Heading1'].spaceBefore = 12; styles['Heading1'].spaceAfter = 6
    styles['Heading2'].fontName = 'Helvetica-Bold'; styles['Heading2'].fontSize = 11; styles['Heading2'].textColor = pdf_secondary_h_color; styles['Heading2'].spaceBefore = 10; styles['Heading2'].spaceAfter = 4
    small_style_pdf = ParagraphStyle(name='SmallPDF', parent=styles['Normal'], fontSize=8, textColor=pdf_text_color)
    small_bold_pdf = ParagraphStyle(name='SmallBoldPDF', parent=small_style_pdf, fontName='Helvetica-Bold')
    res_pass_pdf = ParagraphStyle(name='ResPassPDF', parent=small_style_pdf, textColor=pdf_success_color)
    res_fail_pdf = ParagraphStyle(name='ResFailPDF', parent=small_style_pdf, textColor=pdf_danger_color)
    res_neu_pdf = ParagraphStyle(name='ResNeuPDF', parent=small_style_pdf, textColor=pdf_neutral_color)
    res_warn_pdf = ParagraphStyle(name='ResWarnPDF', parent=small_style_pdf, textColor=pdf_warning_color)
    pdf_bullet_style = ParagraphStyle(name='PdfBulletStyle', parent=styles['Normal'], leftIndent=18, bulletIndent=0, firstLineIndent=0, spaceBefore=2)
    line_style_pdf = ParagraphStyle(name='LineStylePDF', parent=styles['Normal'], fontSize=9, leading=12, spaceAfter=0)
    story: list[object] = []

    # --- PORTADA Y RESUMEN INICIAL ---
    story.append(Paragraph("INFORME DE COMPARACIÓN DE EQUIVALENCIA DE EMPLEOS", ParagraphStyle(name='MainTitle', parent=styles['Heading1'], alignment=TA_CENTER, fontSize=16, textColor=pdf_primary_h_color, spaceAfter=12, leading=20)))
    if params.get('Proceso Selección'): story.append(Paragraph(f"Proceso: {params['Proceso Selección']}", ParagraphStyle(name='SubTitle', parent=styles['Normal'], alignment=TA_CENTER, fontSize=10, spaceAfter=4, textColor=pdf_text_light_color)))
    if params.get('Analista'): story.append(Paragraph(f"Analista: {params['Analista']}", ParagraphStyle(name='SubTitle2', parent=styles['Normal'], alignment=TA_CENTER, fontSize=10, spaceAfter=4, textColor=pdf_text_light_color)))
    if params.get('Profesion Aspirante'): story.append(Paragraph(f"Profesión Aspirante/Servidor: {params['Profesion Aspirante']}", ParagraphStyle(name='SubTitle3', parent=styles['Normal'], alignment=TA_CENTER, fontSize=10, spaceAfter=4, textColor=pdf_text_light_color)))
    tipo_uso_str = str(params.get('Tipo de Uso', 'No especificado')) + (f" (Especificación: {params['Tipo de Uso Otro']})" if params.get('Tipo de Uso Otro') else "")
    story.append(Paragraph(f"Tipo de Uso del Estudio: {tipo_uso_str}", ParagraphStyle(name='SubTitle4', parent=styles['Normal'], alignment=TA_CENTER, fontSize=10, spaceAfter=4, textColor=pdf_text_light_color)))
    story.append(Paragraph(f"Fecha de Generación: {params.get('Fecha')}", ParagraphStyle(name='DateTitle', parent=styles['Normal'], alignment=TA_CENTER, fontSize=10, spaceAfter=18, textColor=pdf_text_light_color)))

    ### MODIFICACIÓN: Perfil completo del empleo base ###
    story.append(Paragraph("1. Información del Empleo Base", styles['Heading1']))
    info_base_data = []
    for key, label in [(ID_COLUMN_NAME, ID_COLUMN_NAME + ":"), ('Código', "Código:"), ('Denominación', "Denominación:"), ('Grado', "Grado:"), ('Nivel Jerárquico', "Nivel Jerárquico:"), ('Orden', "Orden:"), ('Naturaleza Jurídica', "Naturaleza Jurídica:"), ('Salario', "Salario Básico Mensual:")]:
        value = params.get(key, 'N/A');
        if key == 'Salario' and isinstance(value, (int, float)): value = f"${value:,.0f}"
        info_base_data.append([Paragraph(f"<b>{label}</b>", small_bold_pdf), Paragraph(str(value), small_style_pdf)])
    table_base_info_short = Table(info_base_data, colWidths=[2.2*inch, 4.6*inch], hAlign='LEFT', spaceBefore=6)
    table_base_info_short.setStyle(TableStyle([('GRID', (0,0), (-1,-1), 0.5, pdf_grid_color), ('VALIGN', (0,0), (-1,-1), 'TOP'), ('LEFTPADDING', (0,0), (-1,-1), 5), ('RIGHTPADDING', (0,0), (-1,-1), 5)]))
    story.append(table_base_info_short)
    story.append(Spacer(1, 0.1*inch))

    base_text_fields = {
        "Funciones:": 'Funciones Base',
        "Requisitos Estudio:": 'Requisitos Estudio Base',
        "Requisitos Experiencia:": 'Requisitos Experiencia Base',
        "Competencias Laborales:": 'Competencias Laborales Base',
        "Competencias Comportamentales:": 'Competencias Comportamentales Base'
    }
    for label, key in base_text_fields.items():
        text_content = str(params.get(key, ''))
        story.append(Paragraph(f"<b>{label}</b>", small_bold_pdf))
        story.append(Paragraph(text_content if _is_valid_data(text_content) else '<i>(No especificado)</i>', styles['SmallBodyJustify']))
        story.append(Spacer(1, 0.1*inch))
    ### FIN DE LA MODIFICACIÓN ###

    story.append(Spacer(1, 0.1*inch))
    story.append(Paragraph("2. Resumen Global de Comparaciones", styles['Heading1']))

    # Añadir tabla de resumen
    tabla_resumen_pdf_inicio = generar_tabla_resumen_global_pdf(destinos_res_dict, styles, pdf_text_color, pdf_success_color, pdf_danger_color, pdf_neutral_color, pdf_warning_color, pdf_grid_color, pdf_accent_bg_color)
    if tabla_resumen_pdf_inicio:
        story.append(tabla_resumen_pdf_inicio)
        story.append(Spacer(1, 0.1*inch))
        nota_reqs_opcional = "Los Requisitos NO fueron incluidos en la decisión." if not params.get('incluir_requisitos_en_decision', True) else f"Los Requisitos fueron incluidos en la decisión (Pesos: Estd: {params.get('peso_estudio_decision', 0.0):.0%}, C.Lab: {params.get('peso_comp_lab_decision', 0.0):.0%}, C.Comp: {params.get('peso_comp_comp_decision', 0.0):.0%})."
        story.append(Paragraph(f"<i>Nota: La decisión 'Equivalente (Auto)' se basa en Nivel, Salario/Grado, y Funciones (Peso: {params.get('peso_funciones_decision', 0.0):.0%}). {nota_reqs_opcional} La verificación manual es esencial.</i>", small_style_pdf))
    else: story.append(Paragraph("<i>No se pudo generar la tabla resumen global.</i>", small_style_pdf))

    # Añadir gráfico de resumen de puntuaciones
    grafico_puntuaciones_buffer = generar_grafico_resumen_puntuaciones(destinos_res_dict)
    if grafico_puntuaciones_buffer:
        story.append(Spacer(1, 0.2*inch))
        num_destinos = len(destinos_res_dict)
        chart_height = max(1.5, num_destinos * 0.5) * inch
        story.append(ReportlabImage(grafico_puntuaciones_buffer, width=7*inch, height=chart_height, kind='proportional'))

    # --- BUCLE POR CADA DESTINO ---
    for destino_counter, (key_dest_pdf, res_dest_pdf_dict) in enumerate(destinos_res_dict.items()):
        raw_dest_df_row = res_dest_pdf_dict.get('raw_dest', pd.Series(dtype=object))
        story.append(PageBreak())
        story.append(Paragraph(f"{destino_counter + 3}. Comparación con Empleo Destino: {key_dest_pdf}", styles['Heading1']))
        story.append(Paragraph("<i>A continuación, se presenta el análisis de equivalencia automatizado entre el empleo base y este empleo destino, seguido de la sección para la verificación manual de requisitos del aspirante/servidor público.</i>", small_style_pdf))
        story.append(Spacer(1, 0.1*inch))
        story.append(Paragraph("Resumen de Criterios de Equivalencia Automatizada", styles['Heading2']))
        summary_data_pdf = []
        nivel_res_pdf = res_dest_pdf_dict.get('nivel', {})
        summary_data_pdf.append([Paragraph("<b>1. Nivel Jerárquico:</b>", small_bold_pdf), Paragraph(str(nivel_res_pdf.get('mensaje','Error')), res_pass_pdf if nivel_res_pdf.get('pasa') else res_fail_pdf)])
        salario_res_pdf = res_dest_pdf_dict.get('salario', {})
        summary_data_pdf.append([Paragraph("<b>2. Salario / Grado:</b>", small_bold_pdf), Paragraph(str(salario_res_pdf.get('mensaje','Error')), res_pass_pdf if salario_res_pdf.get('pasa') else res_fail_pdf)])
        ref_model_id_for_pdf = res_dest_pdf_dict.get('reference_model_id')
        ref_model_desc_for_pdf = "N/A"
        if ref_model_id_for_pdf and ref_model_id_for_pdf in res_dest_pdf_dict.get('results_per_model', {}):
            ref_model_desc_for_pdf = res_dest_pdf_dict['results_per_model'][ref_model_id_for_pdf].get('model_desc','REF').split(' (')[0]

        reqs_en_decision_pdf = params.get('incluir_requisitos_en_decision', True)

        sim_criteria_pdf = [
            ("Funciones", 'pasa_funciones', 'similitud_funciones', params.get('umbral_decision_funciones', 0.6), 'Funciones Base', 'Funciones', False),
            ("Requisitos Estudio", 'pasa_educacion_decision', 'similitud_educacion', params.get('umbral_decision_requisitos_estudio', 0.7), 'Requisitos Estudio Base', 'Requisitos Estudio', not reqs_en_decision_pdf),
            ("Requisitos Experiencia", None, 'similitud_experiencia', params.get('umbral_decision_requisitos_comp', 0.65), 'Requisitos Experiencia Base', 'Requisitos Experiencia', True),
            ("Competencias Laborales", 'pasa_comp_lab', 'similitud_comp_lab', params.get('umbral_decision_requisitos_comp', 0.65), 'Competencias Laborales Base', 'Competencias Laborales', not reqs_en_decision_pdf),
            ("Competencias Comportamentales", 'pasa_comp_comp', 'similitud_comp_comp', params.get('umbral_decision_requisitos_comp', 0.65), 'Competencias Comportamentales Base', 'Competencias Comportamentales', not reqs_en_decision_pdf)
        ]

        crit_counter = 3
        for label_crit, pasa_key, sim_key, umbral_val, raw_base_k, raw_target_k, es_informativo in sim_criteria_pdf:
            pasa_flag = res_dest_pdf_dict.get(pasa_key) if pasa_key else None
            sim_data_crit = {}
            if ref_model_id_for_pdf and ref_model_id_for_pdf in res_dest_pdf_dict.get('results_per_model', {}):
                sim_data_crit = res_dest_pdf_dict['results_per_model'][ref_model_id_for_pdf].get(sim_key,{})
            msg_crit, style_crit = "Error", res_fail_pdf
            base_has_data_crit, target_has_data_crit = _is_valid_data(str(params.get(raw_base_k, ""))), _is_valid_data(str(raw_dest_df_row.get(raw_target_k, "")))
            if not base_has_data_crit and not target_has_data_crit: msg_crit, style_crit = "No aplica (sin datos válidos en ambos empleos).", res_neu_pdf
            elif not base_has_data_crit or not target_has_data_crit: msg_crit, style_crit = f"Datos incompletos (solo en un empleo).", res_warn_pdf
            elif sim_data_crit.get('error'): msg_crit, style_crit = f"Error cálculo (Mod: {ref_model_desc_for_pdf}): {str(sim_data_crit['error'])[:50]}...", res_fail_pdf
            elif not sim_data_crit.get('metricas'): msg_crit, style_crit = f"Métricas no procesables. {sim_data_crit.get('jerarquia_msg', '')}", res_warn_pdf
            else:
                cob_crit = sim_data_crit.get('metricas', {}).get('cobertura_prom_max_1', 0.0); niv_crit_text, _ = obtener_nivel_similitud(cob_crit)
                msg_crit = f"Cob. E1 x E2: {cob_crit:.1%}. Nivel: {niv_crit_text}. (Umbral Ref: {float(umbral_val):.0%}, Mod: {ref_model_desc_for_pdf})."
                if sim_key == 'similitud_experiencia': msg_crit = f"{sim_data_crit.get('jerarquia_msg', '')} {msg_crit}"
                style_crit = res_warn_pdf if es_informativo else (res_pass_pdf if pasa_flag else res_fail_pdf)
            summary_data_pdf.append([Paragraph(f"<b>{crit_counter}. {label_crit}{' (Info)' if es_informativo else ''}:</b>", small_bold_pdf), Paragraph(msg_crit, style_crit)]); crit_counter+=1

        summary_table_pdf = Table(summary_data_pdf, colWidths=[2.2*inch, 4.6*inch], hAlign='LEFT', spaceBefore=6); summary_table_pdf.setStyle(TableStyle([('GRID', (0,0), (-1,-1), 0.5, pdf_grid_color), ('VALIGN', (0,0), (-1,-1), 'TOP'), ('LEFTPADDING', (0,0), (-1,-1), 5), ('RIGHTPADDING', (0,0), (-1,-1), 5), ('TOPPADDING', (0,0), (-1,-1), 3), ('BOTTOMPADDING', (0,0), (-1,-1), 3)])); story.append(summary_table_pdf)
        story.append(Spacer(1, 0.15*inch))
        es_equiv_auto_pdf = res_dest_pdf_dict.get('es_equivalente', False); equivalence_score_pdf = res_dest_pdf_dict.get('equivalence_score', 0.0)
        texto_fin_auto_pdf = "SÍ SON EQUIVALENTES (AUTOMATIZADO)" if es_equiv_auto_pdf else "NO SON EQUIVALENTES (AUTOMATIZADO)"; estilo_fin_auto_pdf = ParagraphStyle(name='DecisionAutoPDF', parent=styles['Heading2'], alignment=TA_CENTER, textColor=pdf_success_color if es_equiv_auto_pdf else pdf_danger_color, fontSize=12)
        story.append(Paragraph("DECISIÓN AUTOMATIZADA DE EQUIVALENCIA:", ParagraphStyle(name=f'DecisionAutoTitlePDF_{destino_counter}', parent=styles['Heading2'], alignment=TA_CENTER)))
        story.append(Paragraph(texto_fin_auto_pdf, estilo_fin_auto_pdf)); score_color = pdf_success_color if es_equiv_auto_pdf else pdf_danger_color
        story.append(Paragraph(f"Puntuación de Equivalencia (ponderada): {equivalence_score_pdf:.2%}", ParagraphStyle(name='ScorePDF', parent=small_bold_pdf, alignment=TA_CENTER, textColor=score_color)))
        if not es_equiv_auto_pdf:
            razones_auto_pdf = res_dest_pdf_dict.get('razones_no_equivalencia', [])
            if razones_auto_pdf: story.append(Paragraph("<i>Motivos Principales (Automatizados):</i>", small_style_pdf)); [story.append(Paragraph(rz_pdf, pdf_bullet_style, bulletText='•')) for rz_pdf in razones_auto_pdf]
        story.append(Spacer(1, 0.2*inch))
        story.append(Paragraph("Verificación de Cumplimiento de Requisitos Mínimos (Estudio y Experiencia)", styles['Heading2']))
        story.append(Paragraph(f"<b>Profesión del Aspirante/Servidor Público:</b> {params.get('Profesion Aspirante', 'No especificada')}", small_style_pdf)); story.append(Spacer(1, 0.1*inch))
        story.append(Paragraph("<b>Análisis de Cumplimiento (a ser diligenciado por el analista):</b>", small_bold_pdf)); analysis_lines = [Paragraph("_" * 100, line_style_pdf)] * 5; analysis_space = KeepInFrame(6.8*inch, 0.8*inch, analysis_lines); story.append(analysis_space); story.append(Spacer(1, 0.1*inch))

        ### INICIO DE LA CORRECCIÓN DEL CHECKBOX ###
        story.append(Paragraph("<b>Dictamen del Análisis de Requisitos (marcar uno):</b>", small_bold_pdf))

        # Crear un estilo para la tabla del checkbox
        check_box_style = TableStyle([('BOX', (0, 0), (0, 0), 1, pdf_text_color)])

        # Crear instancias separadas para cada checkbox
        check_box_si = Table([['']], colWidths=[9], rowHeights=[9], style=check_box_style)
        check_box_no = Table([['']], colWidths=[9], rowHeights=[9], style=check_box_style)

        # Crear una tabla para alinear los checkboxes y el texto
        dictamen_req_table = Table(
            [[check_box_si, Paragraph("Sí Cumple", small_style_pdf), check_box_no, Paragraph("No Cumple", small_style_pdf)]],
            colWidths=[15, 80, 15, 80],
            style=[('VALIGN', (0,0), (-1,-1), 'MIDDLE')]
        )
        story.append(dictamen_req_table)
        story.append(Spacer(1, 0.2*inch))
        ### FIN DE LA CORRECCIÓN DEL CHECKBOX ###

        if ref_model_id_for_pdf and res_dest_pdf_dict.get('plot_data'):
            story.append(PageBreak()); story.append(Paragraph(f"Análisis Gráfico Detallado para Destino: {key_dest_pdf}", styles['Heading1'])); story.append(Paragraph("<i>Esta sección presenta visualizaciones de las matrices de similitud y los diagramas de conexiones para los aspectos evaluados, usando el Modelo de Referencia.</i>", small_style_pdf))
            for _, (sim_data_plot, titulo_base_plot, _, _, _) in res_dest_pdf_dict['plot_data'].items():
                heatmap_buffer_viz, conex_buffer_viz = sim_data_plot.get('heatmap_buffer'), sim_data_plot.get('conex_buffer')
                if not heatmap_buffer_viz and not conex_buffer_viz: story.append(Paragraph(f"<i>(No hay datos visuales para {titulo_base_plot} o ocurrió un error en su procesamiento)</i>", small_style_pdf)); continue
                story.append(Paragraph(f"<b>{titulo_base_plot}</b> (Ref: {ref_model_desc_for_pdf})", styles['Heading2']))
                if heatmap_buffer_viz:
                    try: story.append(ReportlabImage(heatmap_buffer_viz, width=6.8*inch, height=4.8*inch, kind='proportional')); story.append(Spacer(1, 0.1*inch))
                    except Exception as img_err: story.append(Paragraph(f"<i>Error al incrustar heatmap {titulo_base_plot}: {img_err}</i>", small_style_pdf))
                else: story.append(Paragraph(f"<i>(Heatmap {titulo_base_plot} no disponible)</i>", small_style_pdf))
                if conex_buffer_viz:
                    try: story.append(ReportlabImage(conex_buffer_viz, width=6.5*inch, height=4.5*inch, kind='proportional')); story.append(Spacer(1, 0.1*inch))
                    except Exception as img_err: story.append(Paragraph(f"<i>Error al incrustar diagrama {titulo_base_plot}: {img_err}</i>", small_style_pdf))
                else: story.append(Paragraph(f"<i>(Diagrama {titulo_base_plot} no disponible)</i>", small_style_pdf))

    ### INICIO DE LA CORRECCIÓN DEL CHECKBOX (PÁGINA DE CERTIFICACIÓN) ###
    story.append(PageBreak())
    story.append(Paragraph("CERTIFICACIÓN DEL ESTUDIO TÉCNICO DE EQUIVALENCIA", styles['Heading1']))
    story.append(Spacer(1, 0.2 * inch))

    analista_nombre = params.get('Analista', '__________________')

    declaracion_texto = f"""
    Yo, <b>{analista_nombre}</b>, identificado/a con C.C. No. __________________________,
    en mi calidad de analista responsable, certifico que he revisado el análisis técnico
    automatizado presentado en este documento. Adicionalmente, he realizado la verificación
    manual de los requisitos de estudio y experiencia del servidor/aspirante en relación con
    los empleos destino objeto de este estudio, emitiendo un dictamen para cada uno a continuación.
    """
    story.append(Paragraph(declaracion_texto, styles['BodyJustify']))
    story.append(Spacer(1, 0.3 * inch))

    story.append(Paragraph("<b>DICTAMEN FINAL INDIVIDUAL POR EMPLEO DESTINO</b>", styles['Heading2']))

    check_box_style_cert = TableStyle([('BOX', (0, 0), (0, 0), 1, pdf_text_color)])

    for key_dest, res_dest in destinos_res_dict.items():
        story.append(Spacer(1, 0.15 * inch))

        # Crear instancias de checkbox para esta sección
        check_box_equiv = Table([['']], colWidths=[10], rowHeights=[10], style=check_box_style_cert)
        check_box_no_equiv = Table([['']], colWidths=[10], rowHeights=[10], style=check_box_style_cert)

        # Crear tabla para las opciones del dictamen
        dictamen_options_table = Table([
            [check_box_equiv, Paragraph("SE CONSIDERA EQUIVALENTE", styles['Normal'])],
            [Spacer(1, 5)], # Espacio entre opciones
            [check_box_no_equiv, Paragraph("NO SE CONSIDERA EQUIVALENTE", styles['Normal'])]
        ], colWidths=[0.3 * inch, 6.2 * inch], style=[('VALIGN', (0, 0), (-1, -1), 'TOP')])

        # Usar una tabla principal para mantener el bloque junto
        dictamen_data = [
            [Paragraph(f"<b>Dictamen para Empleo:</b> {truncar_texto(key_dest, 80)}", styles['Normal'])],
            [dictamen_options_table],
            [Spacer(1, 0.1*inch)],
            [Paragraph("<b>Observaciones:</b>", styles['Normal'])],
            [Paragraph("_" * 105, line_style_pdf)],
            [Paragraph("_" * 105, line_style_pdf)],
        ]

        dictamen_table = Table(dictamen_data, colWidths=[6.8*inch])
        dictamen_table.setStyle(TableStyle([
            ('BOX', (0,0), (-1,-1), 1, reportlab_colors.grey),
            ('LEFTPADDING', (0,0), (-1,-1), 6),
            ('RIGHTPADDING', (0,0), (-1,-1), 6),
            ('TOPPADDING', (0,0), (-1,-1), 6),
            ('BOTTOMPADDING', (0,0), (-1,-1), 6),
        ]))
        story.append(dictamen_table)

    story.append(Spacer(1, 0.5 * inch))

    firma_texto = f"""
    <br/><br/>
    ____________________________________<br/>
    <b>Firma del Analista</b><br/>
    <b>Nombre:</b> {analista_nombre}<br/>
    <b>C.C. No.:</b> _________________________
    """
    story.append(Paragraph(firma_texto, styles['Normal']))
    ### FIN DE LA CORRECCIÓN DEL CHECKBOX (PÁGINA DE CERTIFICACIÓN) ###

    # --- FINAL DEL PDF ---
    try:
        with status_area: clear_output(wait=True); print(f"⏳ Generando PDF en memoria: '{pdf_filename}'...")
        doc.build(story, onFirstPage=add_page_bg_light, onLaterPages=add_page_bg_light)
        with open(pdf_filename, 'wb') as f:
            f.write(pdf_buffer.getvalue())
        pdf_buffer.close()
        with status_area: print(f"⬇️ Iniciando descarga del archivo '{pdf_filename}'...")
        files.download(pdf_filename)
        with status_area: clear_output(wait=True); print(f"✅ Informe PDF '{pdf_filename}' generado y descarga iniciada.")
    except Exception as ex_pdf:
        with status_area: clear_output(wait=True); print(f"❌ Error crítico al generar o guardar el PDF: {ex_pdf}");
        logging.error("Error generando PDF", exc_info=True)

def on_exportar_click_excel(b):
    """Genera un Excel de resumen de todas las comparaciones y lo descarga."""
    if not results_data or not results_data.get('results_por_destino'):
        with status_area: clear_output(wait=True); print("⚠️ No hay resultados disponibles para exportar a Excel."); return

    params = results_data.get('params', {})
    destinos_results_all = results_data.get('results_por_destino', {})
    wb = Workbook()
    ws_summary = wb.active; ws_summary.title = "Resumen Equivalencias"
    header = [
        f"{ID_COLUMN_NAME} Base", "Código Base", "Denominación Base", f"{ID_COLUMN_NAME} Destino", "Código Destino", "Denominación Destino",
        "Profesión Aspirante/Servidor", "Tipo de Uso Estudio", "Pasa Nivel", "Pasa Salario/Grado",
        "Requisitos en Decisión?", "Es Equivalente (Autom.)", "Puntuación Eq. (Auto)", "Razones No Equivalencia (Auto)",
    ]
    loaded_model_ids = params.get('loaded_model_ids', [])
    aspects = ['Funciones', 'Estudio', 'Experiencia', 'CompLab', 'CComp']
    for mid in loaded_model_ids:
        model_desc = MODEL_DESCRIPTIONS.get(mid, mid).split(' (')[0]
        for aspect in aspects: header.extend([f"CobProm_{aspect}_{model_desc}", f"Error_{aspect}_{model_desc}"])
        header.append(f"Jerarquia_Exp_{model_desc}")
    ws_summary.append(header)

    header_font = Font(bold=True, color="FFFFFF"); header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
    pass_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid"); fail_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
    info_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid"); na_fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")
    for cell in ws_summary[1]: cell.font = header_font; cell.fill = header_fill; cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

    for res_dest_excel_dict in destinos_results_all.values():
        raw_base = results_data.get('raw_base', pd.Series(dtype=object)); raw_dest = res_dest_excel_dict.get('raw_dest', pd.Series(dtype=object))
        row_data = [
            raw_base.get(ID_COLUMN_NAME), raw_base.get('Código'), raw_base.get('Denominación'),
            raw_dest.get(ID_COLUMN_NAME), raw_dest.get('Código'), raw_dest.get('Denominación'),
            params.get('Profesion Aspirante'), params.get('Tipo de Uso'), res_dest_excel_dict.get('nivel', {}).get('pasa'),
            res_dest_excel_dict.get('salario', {}).get('pasa'), params.get('incluir_requisitos_en_decision'),
            res_dest_excel_dict.get('es_equivalente'), res_dest_excel_dict.get('equivalence_score', 0.0),
            "; ".join(res_dest_excel_dict.get('razones_no_equivalencia', [])),
        ]
        for mid in loaded_model_ids:
            model_results = res_dest_excel_dict.get('results_per_model', {}).get(mid, {})
            aspect_map = {'Funciones': 'similitud_funciones', 'Estudio': 'similitud_educacion', 'Experiencia': 'similitud_experiencia', 'CompLab': 'similitud_comp_lab', 'CComp': 'similitud_comp_comp'}
            for aspect_name, sim_internal_key in aspect_map.items():
                aspect_sim_data = model_results.get(sim_internal_key, {})
                cob_prom = aspect_sim_data.get('metricas', {}).get('cobertura_prom_max_1')
                row_data.extend([cob_prom, aspect_sim_data.get('error')])
            row_data.append(model_results.get('similitud_experiencia', {}).get('jerarquia_msg'))
        ws_summary.append(row_data)

    for row_idx, row in enumerate(ws_summary.iter_rows(min_row=2), start=2):
        for cell in row: cell.alignment = Alignment(horizontal='left', vertical='center', wrap_text=True)
        for col_name, fill_if_true, fill_if_false in [("Pasa Nivel", pass_fill, fail_fill),("Pasa Salario/Grado", pass_fill, fail_fill),("Es Equivalente (Autom.)", pass_fill, fail_fill)]:
            col_idx = header.index(col_name)
            cell = ws_summary.cell(row=row_idx, column=col_idx + 1)
            if cell.value is not None: cell.fill = fill_if_true if cell.value else fill_if_false
            else: cell.fill = na_fill

    base_id_fn_excel = str(params.get(ID_COLUMN_NAME, 'Base')).replace(' ', '_').replace('/', '_')[:30]
    fecha_fn = str(params.get('Fecha', '')).replace('/', '-')
    excel_filename = f"Resumen_Equivalencias_{base_id_fn_excel}_{fecha_fn}.xlsx"

    try:
        with status_area: clear_output(wait=True); print(f"⏳ Generando Excel en memoria: '{excel_filename}'...")
        excel_buffer = io.BytesIO()
        wb.save(excel_buffer)
        with open(excel_filename, 'wb') as f:
            f.write(excel_buffer.getvalue())
        excel_buffer.close()
        with status_area: print(f"⬇️ Iniciando descarga del archivo '{excel_filename}'...")
        files.download(excel_filename)
        with status_area: clear_output(wait=True); print(f"✅ Excel de resumen '{excel_filename}' generado y descarga iniciada.")
    except Exception as e_excel:
        with status_area: clear_output(wait=True); print(f"❌ Error crítico al generar o guardar el Excel: {e_excel}")
        logging.error("Error generando Excel", exc_info=True)


boton_exportar_pdf.on_click(on_exportar_click_pdf)
boton_exportar_excel.on_click(on_exportar_click_excel)


# --- Montar toda la UI en Colab ---
display(widgets.HTML("<link rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css'>"))
display(widgets.HTML(f"""<style>
    .widget-label {{ color: #e0e0e0 !important; }}
    .widget-text input, .widget-textarea textarea, .widget-dropdown select, .widget-select-multiple select {{
        background-color: #3a3a3a !important; color: #f0f0f0 !important; border: 1px solid #555555 !important;
    }}
    .widget-checkbox label {{ color: #e0e0e0 !important; font-size: 0.9em; }}
    .widget-html-content b {{ color: #90caf9; }}
    .widget-button {{ border-radius: 4px; }}
    .jp-OutputArea-output {{ background-color: #212121; color: #e0e0e0; }}
    .output_subarea {{ background-color: #212121 !important; }}
    .p-Accordion-child {{ background-color: #2a2a2a !important; padding: 10px; border-radius: 5px; }}
    .p-Accordion-header {{ background-color: #333333 !important; color: white !important;}}
</style>"""))

titulo_html_widget = widgets.HTML(f"""
<div style="background-color:{AppColors.ACCENT_BACKGROUND.value.hexval()}; border-bottom:2px solid {AppColors.ACCENT_BORDER.value.hexval()}; padding:12px 15px; margin-bottom:15px; border-radius:5px;">
    <h2 style="color:{AppColors.PRIMARY.value.hexval()}; margin:0; font-size:1.7em; font-weight:500;">Analizador de Equivalencia de Empleos</h2>
    <p style="color:{AppColors.TEXT_LIGHT.value.hexval()}; margin:5px 0 0 0; font-size:0.95em;">Cargue su archivo Excel, seleccione empleos base y destino(s) usando '{ID_COLUMN_NAME}', configure parámetros y analice.</p>
</div>
""")

# --- Estructura de UI Mejorada con Acordeón ---
paso1_carga = widgets.VBox([
    upload_btn, lbl_archivo
])
paso2_seleccion = widgets.VBox([
    base_dropdown,
    marcar_todos_destinos_checkbox,
    destinos_select
])

accordion_principal = widgets.Accordion(children=[paso1_carga, paso2_seleccion, accordion_config])
accordion_principal.set_title(0, 'Paso 1: Cargar Archivo')
accordion_principal.set_title(1, 'Paso 2: Seleccionar Empleos')
accordion_principal.set_title(2, 'Paso 3: Configurar Análisis')
accordion_principal.selected_index = 0

output_accordion = widgets.Accordion(children=[], selected_index=None)

ui_completa = widgets.VBox([
    titulo_html_widget,
    accordion_principal,
    botones_box,
    widgets.HTML(value=f"<hr style='border-color:{AppColors.ACCENT_BORDER.value.hexval()}; margin:15px 0;'/>"),
    widgets.HTML(value=f"<h3 style='color:{AppColors.SECONDARY.value.hexval()};'>Mensajes de Estado:</h3>"),
    status_area,
    widgets.HTML(value=f"<hr style='border-color:{AppColors.ACCENT_BORDER.value.hexval()}; margin:15px 0;'/>"),
    widgets.HTML(value=f"<h3 style='color:{AppColors.SECONDARY.value.hexval()};'>Resultados del Análisis:</h3>"),
    output_accordion
], layout={'padding': '10px', 'background_color': '#1e1e1e'})

display(ui_completa)

HTML(value="<link rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-aw…

HTML(value='<style>\n    .widget-label { color: #e0e0e0 !important; }\n    .widget-text input, .widget-textare…

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]